###### ## Arquivo para gerar o indicador PrEP:Vinculados por município     
                                                                      
### Versão: 1.06
                                                                 
Descrição:          Este script gera automaticamente uma lista com todos os municípios e o respectivo valor do indicador "PrEP:Novos_Vinculados".
                                                          
### Autor:              Assessoria de Monitoramento e Avaliação (AMA)    
                                                                      
Data de Criação:    21/12/2023                                        
                                                                   
Última Atualização: 21/02/2025                              
                                                                     
Sobre versão: Coloca todos os municípios nas tabelas. gera tabela BI para Painel de Eliminação, revisado em 06/05/2025.
                                                                       

## Configuração do Ambiente

In [1]:
from datetime import datetime, date
from datetime import timedelta
from dateutil import parser
import io
from io import BytesIO
import locale
import math
import numpy as np
import os
import sys
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl import load_workbook
import pandas as pd
import shutil
import time
import xlsxwriter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import kendalltau
from scipy.stats import ttest_1samp
import pymannkendall as mk

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [3]:
sys.path.append("//SAP109/Bancos AMA/Arquivos Atuais/Programas Atuais/Python/Programas Ativos/Monitoramento Clinico")
# Importa as funções próprias
import funcoes_gerais as fg
import organizacao_bases_hiv as org
import organizacao_bases_prep as prep

In [4]:
cores = ['blue',
        'red',
        'green',
        'purple',
        'orange',
        'cyan',
        'magenta',
        'yellow',
        'brown',
        'lime',
        'teal',
        'coral',
        'indigo',
        'gold',
        'violet',
        'turquoise',
        'khaki',
        'orchid',
        'sienna',
        'steelblue',
        'peru',
        'mediumseagreen',
        'crimson',
        'darkorange',
        'darkcyan',
        'darkviolet',
        'darkslategrey']

In [5]:
## ATENÇÃO! Nenhuma célula exige entrada manual. Digite a data de fechamento do banco quando perguntado.

hoje = pd.to_datetime(input("Digite a data de fechamento do banco no formato aaaa-mm-dd \n").strip())
hoje = hoje.normalize()
hoje_str = hoje.strftime('%Y-%m-%d')

ano = hoje.year
mes = hoje.month

mes_nome = fg.mes_nome(hoje)

## Ajuste o ano de corte se necessário
ano_corte = ano - 10

Digite a data de fechamento do banco no formato aaaa-mm-dd 
 2025-12-31


In [6]:
hoje2_dt = pd.to_datetime(hoje)

# Create a dictionary for month names in Brazilian Portuguese
month_names_pt_br = {
    'Jan': 'Jan',
    'Feb': 'Fev',
    'Mar': 'Mar',
    'Apr': 'Abr',
    'May': 'Mai',
    'Jun': 'Jun',
    'Jul': 'Jul',
    'Aug': 'Ago',
    'Sep': 'Set',
    'Oct': 'Out',
    'Nov': 'Nov',
    'Dec': 'Dez'
}

# Format 'hoje2' to the desired format
hoje2 = f"{month_names_pt_br[hoje2_dt.strftime('%b')].lower()}/{hoje2_dt.strftime('%Y')}"

print(hoje2)

dez/2025


## Novos Vinculados nos últimos 6 meses

In [7]:
DF = pd.read_parquet("//SAP109/Bancos AMA/Arquivos Atuais/Bancos Atuais HIV/Mensais/PVHA_prim_ult.parquet")

In [8]:
Pop_ibge = (
    DF[['Populacao_resid', 'codigo_ibge_resid']]
    .drop_duplicates(subset='codigo_ibge_resid')
    .dropna(subset=['codigo_ibge_resid'])
    .copy()
)

Pop_ibge['Populacao_resid'] = Pop_ibge['Populacao_resid'].astype(int)
Pop_ibge['codigo_ibge_resid'] = Pop_ibge['codigo_ibge_resid'].astype(int)

In [9]:
IBGE = pd.read_excel('M:\Arquivos Atuais\Tabelas IBGE/Tabela_IBGE_UF e Municípios.xlsx')
IBGE

,Região,UF,Nome_UF,codigo_ibge_resid,nome_mun
0,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste
1,Norte,11,Rondônia,1100379,Alto Alegre dos Parecis
2,Norte,11,Rondônia,1100403,Alto Paraíso
3,Norte,11,Rondônia,1100346,Alvorada D'Oeste
4,Norte,11,Rondônia,1100023,Ariquemes
...,...,...,...,...,...
5565,Centro-Oeste,52,Goiás,5222005,Vianópolis
5566,Centro-Oeste,52,Goiás,5222054,Vicentinópolis
5567,Centro-Oeste,52,Goiás,5222203,Vila Boa
5568,Centro-Oeste,52,Goiás,5222302,Vila Propício


#### Define os pontos mensais de soma dos vinculados nos 6 meses anteriores

In [10]:
DF["Menor Data"] = pd.to_datetime(DF["data_min"], errors = "coerce")
DF['ano_menor']= DF['Menor Data'].dt.year
DF['mes_menor']= DF['Menor Data'].dt.month

In [11]:
DF = DF.rename(columns = {"reg_res":"regiao_Res", "uf_res":"UF_Res"})

In [12]:
DF['capital_res'] = DF['capital_res'].replace('Porte Alegre', 'Porto Alegre')

In [13]:
DF = DF.fillna(np.nan)

In [14]:
DF

,Cod_unificado,data_min,origem_data_min_x,data_obito,data_nascimento,Sexo_cat,Raca_cat,data_registro_min_x,origem_data_registro_min_x,PVHA,Hoje,Raca_cat2,ano_min,ano_obito,Idade_data_min,Idade_vinc_cat,Idade_vinc_cat_anterior,Idade_Hoje,Idade_atual,codigo_paciente,uf_residencia,acomp_medico,cd_pais,co_orientacao_sexual,co_genero,st_paciente_x,Escol_num,Genero_cat,Orientacao_cat,st_num_x,Pop_genero,pais,codigo_udm_prim,nm_udm_prim,tp_servico_atendimento_prim,cod_ibge_udm_prim,st_pub_priv_prim,data_dispensa_prim,esquema_prim,esquema_forma_prim,duracao_prim,cd_crm_prim,uf_crm_prim,ano_disp_prim,mesN_disp_prim,mes_disp_prim,mes_ano_prim,duracao_sum_prim,Duracao_cat_prim,esquema_AMA_prim,Classe_Med_prim,Classe_Med_Count_prim,esquema_cat_prim,data_esquemaAMA_Prim_x,cod_ibge_solicitante_cd4_prim,cd_inst_sol_cd4_prim,nm_inst_sol_cd4_prim,tipo_inst_sol_cd4_prim,data_solicitacao_cd4_prim,data_hora_coleta_cd4_prim,contagem_cd4_prim,ano_coleta_cd4_prim,mesN_coleta_cd4_prim,mes_coleta_cd4_prim,CD4_cat_prim,CD4_cat200_prim,CD4_cat350_prim,cod_ibge_solicitante_cv_prim,cd_inst_sol_cv_prim,nm_inst_sol_cv_prim,tipo_inst_sol_cv_prim,data_solicitacao_cv_prim,data_hora_coleta_cv_prim,copias_prim,comentario_copias_prim,ano_coleta_cv_prim,mesN_coleta_cv_prim,mes_coleta_cv_prim,N_exames_ano_cv_prim,CV_detec_count_prim,CV_5000_count_prim,CV_cat_prim,CV_cat1000_prim,CV_cat50_prim,CV_cat200_prim,CV_cat500_prim,ID_MUNICIP_Ad,DT_NOTIFIC_Ad,ID_MN_RESI_Ad,Escol_num_Ad,ID_MN_RESI_cong,DT_NOTIFIC_cong,ID_MUNICIP_cong,Escol_num_cong,ID_MUNICIP_gest,DT_NOTIFIC_gest,ID_MN_RESI_gest,Escol_num_gest,ID_MUNICIP_cr,DT_NOTIFIC_cr,ID_MN_RESI_cr,Escol_num_cr,CODMUNCART,CODMUNOCOR,CODMUNRES,Escol_cat,cod_ibge6_res,capital_res,UF_Res,regiao_Res,Nome_mun_resid,Populacao_resid,codigo_ibge_resid,nome_inst_exames_prim,uf_inst_prim,uf_inst_completo_prim,reg_inst_prim,cod_ibge6_exame_prim,ibge_inst_exames_prim,Nome_mun_exame_prim,Populacao_exame_prim,data_min_exame,origem_data_min_exame,ano_min_exame,Dias_diag_TARV,Dias_diag_TARV_cat,Dias_exame_TARV,Dias_exame_TARV_cat,Nome_mun_exame_30+_prim,Cod_ibge_insti_sol_30+_prim,nome_inst_exames_30+_prim,origem_data_min_y,data_registro_min_y,origem_data_registro_min_y,st_paciente_y,st_num_y,codigo_udm_ult,nm_udm_ult,tp_servico_atendimento_ult,cod_ibge_udm_ult,st_pub_priv_ult,data_dispensa_ult,esquema_ult,esquema_forma_ult,duracao_ult,cd_crm_ult,uf_crm_ult,ano_disp_ult,mesN_disp_ult,mes_disp_ult,mes_ano_ult,duracao_sum_ult,dt_lim_prox_disp_ult,dt_real_disp_seguinte_ult,Atraso_ult,Abandono_sum_ult,N_dispensas_ult,Atraso_sum_ult,Atraso_cat_ult,Duracao_cat_ult,esquema_AMA_ult,Classe_Med_ult,Classe_Med_Count_ult,esquema_cat_ult,data_esquemaAMA_Prim_y,cod_ibge_solicitante_cd4_ult,cd_inst_sol_cd4_ult,nm_inst_sol_cd4_ult,tipo_inst_sol_cd4_ult,data_solicitacao_cd4_ult,data_hora_coleta_cd4_ult,contagem_cd4_ult,ano_coleta_cd4_ult,mesN_coleta_cd4_ult,mes_coleta_cd4_ult,N_exames_ano_cd4_ult,CD4_cat_ult,CD4_cat200_ult,CD4_cat350_ult,cod_ibge_solicitante_cv_ult,cd_inst_sol_cv_ult,nm_inst_sol_cv_ult,tipo_inst_sol_cv_ult,data_solicitacao_cv_ult,data_hora_coleta_cv_ult,copias_ult,comentario_copias_ult,ano_coleta_cv_ult,mesN_coleta_cv_ult,mes_coleta_cv_ult,N_exames_ano_cv_ult,CV_detec_count_ult,CV_5000_count_ult,CV_cat_ult,CV_cat1000_ult,CV_cat50_ult,CV_cat200_ult,CV_cat500_ult,Idade_data_dispensa_ult,Idade_data_dispensa_cat_ult,Idade_data_hora_coleta_cv_ult,Idade_data_hora_coleta_cv_cat_ult,Idade_data_hora_coleta_cd4_ult,Idade_data_hora_coleta_cd4_cat_ult,Idade_cascata_ult,Idade_Spec_crianca_ult,Idade_Spec1_ult,Idade_Spec2_ult,nome_inst_exames_ult,uf_inst_ult,uf_inst_completo_ult,reg_inst_ult,cod_ibge6_exame_ult,ibge_inst_exames_ult,Nome_mun_exame_ult,Populacao_exame_ult,Atraso_ano_ult,Status_ano_ult,Vinculado_ano_ult,N_PVHA_CV_ano_Instituicao_ult,Grupo_instituicao_ult,Deteccao_CV50_ult,Deteccao_CV200_ult,Deteccao_CV1000_ult,ultimo_evento,ultimo_vinculo,Vinculado_5anos_ult,Vinculado_10anos_ult,Nome_mun_exame_50+_ult,Cod_ibge_insti_sol_50+_ult,nome_inst_e

In [15]:
Vinculados = pd.DataFrame([])

for ano_seq in range(2022, ano + 1):
    for mes_seq in range(1, 13):

        # Define o dia de análise e exatamente um ano antes.
        last_day = pd.Timestamp(year=ano_seq, month=mes_seq, day=1) + pd.offsets.MonthEnd(0)
        start_date = last_day - pd.offsets.DateOffset(months=6)  # 6 meses antes
        
        # Considera apenas as pessoas apareceram nos últimos 6 meses
        df_periodo = DF[(DF['Menor Data'] > start_date) &
                        (DF['Menor Data'] <= last_day)].copy()

        df_periodo.sort_values(by = "Menor Data", ascending = False, inplace = True)
        df_periodo = df_periodo.drop_duplicates(['Cod_unificado'], keep = "first").copy()        
      

        # Adiciona uma nova linha para cada capital ao DataFrame EmPrEP_Serie_Historica
        for cap in df_periodo['capital_res'].unique():
            Vinculados.at[cap, f"{mes_seq}_{ano_seq}"] = df_periodo[df_periodo['capital_res'] == cap]["Cod_unificado"].nunique()   
        
        # Adiciona uma nova linha para cada UF ao DataFrame EmPrEP_Serie_Historica
        for uf in df_periodo['UF_Res'].unique():
            Vinculados.at[uf, f"{mes_seq}_{ano_seq}"] = df_periodo[df_periodo['UF_Res'] == uf]["Cod_unificado"].nunique()      

        # Adiciona uma nova linha para cada Região ao DataFrame EmPrEP_Serie_Historica
        for regiao in df_periodo['regiao_Res'].unique():
            Vinculados.at[regiao, f"{mes_seq}_{ano_seq}"] = df_periodo[df_periodo['regiao_Res'] == regiao]["Cod_unificado"].nunique()                  
            
        Vinculados.at["Brasil",f"{mes_seq}_{ano_seq}"] = df_periodo["Cod_unificado"].nunique()
        
        if (ano_seq == ano) and (mes_seq == mes):
            break

        
Vinculados

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
Manaus,1041.0,1026.0,966.0,954.0,965.0,998.0,1003.0,997.0,983.0,991.0,999.0,952.0,962.0,939.0,947.0,951.0,914.0,934.0,928.0,982.0,969.0,964.0,996.0,981.0,1007.0,1003.0,983.0,1032.0,1016.0,1025.0,1010.0,1025.0,1035.0,1032.0,1016.0,1016.0,1002.0,1032.0,1021.0,985.0,974.0,967.0,977.0,928.0,884.0,894.0,900.0,875.0
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maceió,309.0,300.0,293.0,293.0,280.0,278.0,279.0,290.0,288.0,291.0,279.0,278.0,297.0,283.0,283.0,283.0,302.0,311.0,308.0,337.0,333.0,333.0,324.0,312.0,297.0,288.0,298.0,308.0,321.0,322.0,328.0,328.0,328.0,324.0,321.0,327.0,337.0,352.0,327.0,333.0,319.0,310.0,310.0,312.0,323.0,315.0,320.0,320.0
Rio de Janeiro,1789.0,1858.0,1894.0,1907.0,1911.0,1953.0,1967.0,1977.0,1911.0,1858.0,1827.0,1844.0,1871.0,1854.0,1889.0,1882.0,1930.0,1904.0,1843.0,1838.0,1807.0,1832.0,1823.0,1819.0,1891.0,1904.0,1892.0,1900.0,1923.0,1903.0,1899.0,1929.0,1895.0,1931.0,1917.0,1934.0,1911.0,1960.0,1914.0,1836.0,1785.0,1745.0,1739.0,1682.0,1721.0,1795.0,1795.0,1799.0
Curitiba,388.0,376.0,408.0,397.0,411.0,417.0,432.0,440.0,430.0,443.0,405.0,405.0,408.0,406.0,407.0,418.0,462.0,472.0,467.0,496.0,492.0,495.0,462.0,439.0,452.0,427.0,412.0,414.0,429.0,432.0,426.0,443.0,434.0,448.0,427.0,416.0,418.0,405.0,410.0,387.0,409.0,429.0,414.0,428.0,404.0,415.0,392.0,368.0
São Luís,344.0,349.0,349.0,350.0,355.0,366.0,368.0,371.0,349.0,347.0,339.0,334.0,357.0,357.0,381.0,382.0,387.0,376.0,367.0,381.0,350.0,347.0,341.0,345.0,337.0,348.0,354.0,376.0,371.0,374.0,375.0,367.0,360.0,354.0,340.0,344.0,356.0,352.0,356.0,350.0,351.0,341.0,341.0,347.0,366.0,376.0,376.0,367.0
Porto Velho,148.0,143.0,134.0,132.0,152.0,148.0,161.0,177.0,164.0,164.0,156.0,163.0,160.0,179.0,189.0,184.0,189.0,185.0,193.0,184.0,172.0,174.0,178.0,178.0,160.0,157.0,156.0,153.0,146.0,159.0,172.0,178.0,183.0,181.0,195.0,181.0,182.0,178.0,173.0,193.0,196.0,188.0,179.0,170.0,176.0,152.0,129.0,125.0
João Pessoa,199.0,210.0,187.0,187.0,197.0,197.0,187.0,188.0,213.0,218.0,198.0,213.0,237.0,242.0,242.0,239.0,262.0,257.0,268.0,272.0,262.0,278.0,270.0,270.0,278.0,284.0,279.0,289.0,297.0,297.0,277.0,269.0,276.0,282.0,278.0,282.0,303.0,312.0,306.0,295.0,292.0,284.0,271.0,267.0,252.0,242.0,236.0,232.0
Fortaleza,745.0,744.0,740.0,723.0,720.0,686.0,661.0,676.0,701.0,709.0,722.0,752.0,808.0,801.0,757.0,748.0,742.0,718.0,695.0,724.0,726.0,751.0,737.0,735.0,734.0,731.0,689.0,677.0,667.0,694.0,716.0,691.0,773.0,778.0,755.0,781.0,738.0,762.0,685.0,709.0,722.0,745.0,714.0,722.0,765.0,731.0,739.0,632.0
Goiânia,420.0,426.0,412.0,418.0,411.0,392.0,396.0,404.0,418.0,408.0,418.0,419.0,441.0,443.0,420.0,424.0,429.0,429.0,411.0,415.0,398.0,411.0,431.0,432.0,439.0,422.0,441.0,446.0,408.0,419.0,421.0,432.0,416.0,413.0,405.0,387.0,401.0,416.0,407.0,390.0,395.0,392.0,359.0,320.0,338.0,335.0,333.0,360.0


In [16]:
Vinculados_mun2 = pd.DataFrame([])

for ano_seq in range(2022, ano + 1):
    for mes_seq in range(1, 13):

        # ultimo dia do mes
        last_day = pd.Timestamp(year=ano_seq, month=mes_seq, day=1) + pd.offsets.MonthEnd(0)
        start_date = last_day - pd.offsets.DateOffset(months=6)

        # filtra periodo de 6 meses
        df_periodo = DF[(DF["Menor Data"] > start_date) &
                        (DF["Menor Data"] <= last_day)].copy()

        df_periodo.sort_values(by="Menor Data", ascending=False, inplace=True)
        df_periodo = df_periodo.drop_duplicates(["Cod_unificado"], keep="first").copy()

        # agrupa por municipio
        grupo_mun = (
            df_periodo
            .groupby(["codigo_ibge_resid", "Nome_mun_resid"])["Cod_unificado"]
            .nunique()
        )

        # grava em Vinculados_mun
        for (cod_ibge, nome_mun), valor in grupo_mun.items():
            Vinculados_mun2.at[cod_ibge, f"{mes_seq}_{ano_seq}"] = valor

        if (ano_seq == ano) and (mes_seq == mes):
            break

# se quiser garantir ordem crescente do index (codigo_ibge_resid)
Vinculados_mun2 = Vinculados_mun2.sort_index()

Vinculados_mun2

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
1100015,NaN,1.0,1.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,NaN,NaN,3.0,3.0,3.0,3.0,3.0,3.0,NaN,1.0,1.0,2.0,5.0,5.0,5.0,4.0,5.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,3.0,3.0
1100023,18.0,17.0,19.0,20.0,20.0,14.0,14.0,12.0,11.0,12.0,15.0,14.0,16.0,20.0,22.0,21.0,25.0,29.0,29.0,27.0,26.0,28.0,25.0,26.0,26.0,28.0,32.0,32.0,28.0,28.0,28.0,30.0,28.0,29.0,33.0,31.0,28.0,20.0,17.0,16.0,15.0,14.0,15.0,17.0,19.0,19.0,20.0,18.0
1100031,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100049,22.0,20.0,18.0,17.0,13.0,10.0,10.0,12.0,9.0,11.0,10.0,12.0,13.0,12.0,13.0,10.0,10.0,11.0,16.0,16.0,20.0,21.0,22.0,21.0,18.0,26.0,22.0,24.0,22.0,23.0,24.0,18.0,16.0,13.0,13.0,10.0,6.0,9.0,11.0,13.0,15.0,16.0,17.0,14.0,14.0,14.0,12.0,11.0
1100056,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,4.0,6.0,6.0,7.0,5.0,5.0,4.0,2.0,2.0,3.0,2.0,2.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,3.0,4.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5221908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5222005,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5222054,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,2.0,2.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5222302,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,4.0,4.0,4.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Define o período dos últimos 6 meses (para a lista de municipios)

In [17]:
data_inicio = pd.to_datetime(hoje) - pd.offsets.DateOffset(months=6)
DF6 = DF[(DF['Menor Data'] > data_inicio) & (DF['Menor Data'] <= hoje)].copy()
DF6.shape

(30495, 243)

In [18]:
pd.DataFrame(DF6[['ano_menor',"mes_menor"]].value_counts().sort_index())

count
ano_menor mes_menor       
2025      7           5645
          8           5070
          9           5028
          10          5511
          11          4880
          12          4361

In [19]:
pd.DataFrame(DF6[['regiao_Res',"UF_Res"]].value_counts(dropna = False).sort_index())

count
regiao_Res   UF_Res       
Centro-Oeste DF        448
             GO       1069
             MS        569
             MT        673
Nordeste     AL        554
             BA       1857
             CE       1378
             MA       1112
             PB        445
             PE       1378
             PI        411
             RN        512
             SE        267
Norte        AC        104
             AM       1138
             AP        215
             PA       1821
             RO        245
             RR        173
             TO        205
Sudeste      ES        697
             MG       2103
             RJ       3437
             SP       4924
Sul          PR       1450
             RS       1902
             SC       1408

In [20]:
Vinculados_Mun = pd.DataFrame(
    DF6[['regiao_Res', 'UF_Res', 'codigo_ibge_resid']].value_counts(dropna=False).sort_index()
)
Vinculados_Mun.rename(columns={"count": "Vinculados"}, inplace=True)
Vinculados_Mun.reset_index(inplace=True)

Vinculados_Mun['codigo_ibge_resid'] = Vinculados_Mun['codigo_ibge_resid'].astype('Int64')


In [21]:
IBGE['codigo_ibge_resid'] = IBGE['codigo_ibge_resid'].astype(str)
Vinculados_Mun['codigo_ibge_resid'] = Vinculados_Mun['codigo_ibge_resid'].astype(str)

IBGE2 = IBGE.merge(
    Vinculados_Mun[['codigo_ibge_resid', 'Vinculados']],
    on='codigo_ibge_resid',
    how='left'
)
IBGE2['Vinculados'] = IBGE2['Vinculados'].fillna(0).astype(int)
IBGE2

,Região,UF,Nome_UF,codigo_ibge_resid,nome_mun,Vinculados
0,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,3
1,Norte,11,Rondônia,1100379,Alto Alegre dos Parecis,0
2,Norte,11,Rondônia,1100403,Alto Paraíso,1
3,Norte,11,Rondônia,1100346,Alvorada D'Oeste,0
4,Norte,11,Rondônia,1100023,Ariquemes,18
...,...,...,...,...,...,...
5565,Centro-Oeste,52,Goiás,5222005,Vianópolis,0
5566,Centro-Oeste,52,Goiás,5222054,Vicentinópolis,0
5567,Centro-Oeste,52,Goiás,5222203,Vila Boa,0
5568,Centro-Oeste,52,Goiás,5222302,Vila Propício,0


## Pessoas em PrEP

In [22]:
Cad, Disp,= prep.carregar_bases(hoje,
                             Cad = True,
                             Disp = True,
)

//SAP109/Bancos AMA\Arquivos Atuais/Programas Atuais/Python/Programas Ativos/Monitoramento Clinico\organizacao_bases_prep.py:102: DtypeWarning: Columns (6,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{caminho}/{base}.txt", sep="\t", names=colunas,


In [23]:
# Carregar o banco de dados cadastro_hiv_uni.csv
cadastro_path = r"V:\\2025\\Monitoramento e Avaliação\\COMPARTILHADO\\AMA - Banco de Dados\\AMA-VIP\\Bancos Compartilhados HIV\\Cadastro.csv"
cadastro_HIV = pd.read_csv(cadastro_path, sep=";", encoding="latin-1", low_memory=False)

In [24]:
# Carregar o banco de dados PVHA.csv
PVHA_path = r"V:\\2025\\Monitoramento e Avaliação\\COMPARTILHADO\\AMA - Banco de Dados\\AMA-VIP\\Bancos Compartilhados HIV\\PVHA.csv"
PVHA_tb = pd.read_csv(PVHA_path, sep=";", encoding="latin-1", low_memory=False)

In [25]:
PVHA_tb

,Cod_unificado,data_min,origem_data_min,data_obito,data_nascimento,Sexo_cat,Raca_cat,data_registro_min,origem_data_registro_min,PVHA
0,100000000,1997-08-21,data_min_sinan,2006-07-27,1975-05-03,Mulher,Parda,1999-04-14,DT_NOTIFIC_sinan,Sim
1,100000001,1996-01-20,data_min_sinan,NaN,1959-11-17,Homem,NaN,1997-03-06,DT_NOTIFIC_sinan,Sim
2,100000002,1999-01-12,data_min_sinan,NaN,1971-05-22,Homem,Parda,1999-01-19,DT_NOTIFIC_sinan,Sim
3,100000003,1999-09-14,data_min_sinan,2003-03-13,1942-10-08,Homem,Parda,1999-09-14,DT_NOTIFIC_sinan,Sim
4,100000004,1997-05-20,data_min_sinan,NaN,1960-05-04,Homem,Parda,1998-12-28,DT_NOTIFIC_sinan,Sim
...,...,...,...,...,...,...,...,...,...,...
1695047,800045456,2025-12-16,data_dispensa,NaN,2000-03-27,Homem,Parda,2025-12-16,data_dispensa,Sim
1695048,800045469,2025-12-18,data_dispensa,NaN,1995-03-01,Homem,Preta,2025-12-18,data_dispensa,Sim
1695049,800045475,2025-12-19,data_dispensa,NaN,1999-06-28,Homem,Branca,2025-12-19,data_dispensa,Sim
1695050,800045485,2025-12-23,data_dispensa,NaN,1998-11-16,Homem,Parda,2025-12-23,data_dispensa,Sim


In [26]:
# Trabalhando as datas

Disp['data_dispensa'] = pd.to_datetime(Disp['data_dispensa'],errors='coerce')
Disp['dt_disp'] = Disp['data_dispensa'].dt.normalize()
Disp['ano_disp']= Disp['dt_disp'].dt.year
Disp['mesN_disp']= Disp['dt_disp'].dt.month
Disp['mes_disp']= Disp['dt_disp'].dt.month_name().str[:3]

# Filtre o DataFrame para manter apenas as linhas com datas em ou antes da 'data_fechamento'
Disp = Disp[(Disp['dt_disp'] <= hoje) & (Disp['ano_disp'] >= 2018)].copy()


# Define a mapping from English month names to Portuguese abbreviations
month_mapping = {
    'Jan': 'Jan',
    'Feb': 'Fev',
    'Mar': 'Mar',
    'Apr': 'Abr',
    'May': 'Mai',
    'Jun': 'Jun',
    'Jul': 'Jul',
    'Aug': 'Ago',
    'Sep': 'Set',
    'Oct': 'Out',
    'Nov': 'Nov',
    'Dec': 'Dez',
}

# Convert the 'dt_disp' column to the month abbreviation and map it to the Portuguese version
Disp['mes_disp'] = Disp['dt_disp'].dt.month_name().str[:3].map(month_mapping)

# Soma a duração de cod_pac e dt_disp iguais.
Disp['duracao_sum'] = Disp.groupby(['codigo_pac_eleito', 'dt_disp'])['duracao'].transform('sum')

# ordenar por cod_pac e dt_disp
Disp = Disp.sort_values(['codigo_pac_eleito', 'dt_disp'],
              ascending = [True, False])

# retira duplicidade de data da dispensa e cria novo banco "Disp_semdupl".
Disp_semdupl = Disp.drop_duplicates(subset=['codigo_pac_eleito', 'dt_disp']).copy()

# ordenar por dt_disp
Disp_semdupl.sort_values(by = "dt_disp", inplace = True)

# Padronizar nomes de colunas
Disp_semdupl = Disp_semdupl.reset_index()

cadastro_HIV.columns = cadastro_HIV.columns.str.strip()
Disp_semdupl.columns = Disp_semdupl.columns.str.strip()

# Fazer o merge novamente
Disp_semdupl = Disp_semdupl.merge(
    cadastro_HIV[['codigo_paciente', 'Cod_unificado']],
    on='codigo_paciente',
    how='left'
)

# Converter mantendo NaNs
Disp_semdupl['Cod_unificado'] = Disp_semdupl['Cod_unificado'].astype('Int64')


In [27]:
# Primeiro, selecionar apenas as colunas desejadas de PVHA_tb
PVHA_selecionado = PVHA_tb[['Cod_unificado', 'data_min', 'data_obito', 'PVHA']].copy()

# Renomear 'data_min' para 'data_min_PVHA'
PVHA_selecionado = PVHA_selecionado.rename(columns={'data_min': 'data_min_PVHA'})

# Fazer o merge com base em Cod_unificado
Disp_semdupl = Disp_semdupl.merge(
    PVHA_selecionado,
    on='Cod_unificado',
    how='left'
)
Disp_semdupl = Disp_semdupl[Disp_semdupl['data_obito'].isna()]

In [28]:

# Cria a coluna com a data da dispensa seguinte
Disp_semdupl.sort_values(by = ['codigo_pac_eleito',"dt_disp"], inplace = True, ascending = [True,False])
Disp_semdupl.reset_index(drop = True, inplace = True)
Disp_semdupl['dt_disp_seguinte'] = Disp_semdupl.groupby('codigo_pac_eleito')['dt_disp'].shift(1)

# calcula as datas de primeira e última dispensa (dt_disp_min e dt_disp_max) da vida
Disp_semdupl["dt_disp_min"] = Disp_semdupl["codigo_pac_eleito"].map(Disp_semdupl.groupby(["codigo_pac_eleito"])["dt_disp"].min())
Disp_semdupl["dt_disp_max"] = Disp_semdupl["codigo_pac_eleito"].map(Disp_semdupl.groupby(["codigo_pac_eleito"])["dt_disp"].max())

# Cria as colunas do mes_pri_disp, mes_ult_disp, ano_pri_disp, ano_ult_disp
Disp_semdupl['ano_pri_disp']= Disp_semdupl['dt_disp_min'].dt.year
Disp_semdupl['mesN_pri_disp']= Disp_semdupl['dt_disp_min'].dt.month
Disp_semdupl['mes_pri_disp']= Disp_semdupl['dt_disp_min'].dt.month_name().str[:3]

Disp_semdupl['ano_ult_disp']= Disp_semdupl['dt_disp_max'].dt.year
Disp_semdupl['mesN_ult_disp']= Disp_semdupl['dt_disp_max'].dt.month
Disp_semdupl['mes_ult_disp']= Disp_semdupl['dt_disp_max'].dt.month_name().str[:3]

# Convert the "dt_disp" column to a datetime format
Disp_semdupl['dt_disp'] = pd.to_datetime(Disp_semdupl['dt_disp'])

# Create a new column "valid_until" that represents the date until which each dispensation is valid
Disp_semdupl['valid_until'] = Disp_semdupl['dt_disp'] + pd.to_timedelta(Disp_semdupl['duracao_sum'] * 1.4, unit='D')

# Initialize an empty DataFrame to hold the results
EmPrEP_monthly_sample = pd.DataFrame(columns=['Year', 'Month', 'Em PrEP', 'Descontinuados'])

# For each month from January 2018 to final DataBase
for year in range(2018, ano + 1):
    for month in range(1, 13):

        # Define the start and end of the period
        last_day = pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(0)  # Get the last day of the current month
        start_date = last_day - pd.offsets.DateOffset(years=1)  # One year before
        end_date = last_day
        
        # Filter the data to consider only dispensations that happened in the past 12 months
        past_year_dispensations = Disp_semdupl[(Disp_semdupl['dt_disp'] > start_date) & (Disp_semdupl['dt_disp'] <= end_date)].copy()
        past_year_dispensations.sort_values(by="dt_disp", ascending=False, inplace=True)
        past_year_dispensations = past_year_dispensations.drop_duplicates(['codigo_pac_eleito'], keep="first").copy()
        
        # Check if the 'valid_until' date is later than the end of the month
        valid_dispensations = past_year_dispensations[past_year_dispensations['valid_until'] >= end_date].copy()

        # Se a data for depois da database, salva o Em PrEP Atual 
        if year == ano and month == int(mes):
            past_year_dispensations["Disp_Ultimos_12m"] = 'Teve dispensação nos últimos 12 meses'
            valid_dispensations["EmPrEP_Atual"] = "Em PrEP atualmente"
            Disp_semdupl = pd.merge(Disp_semdupl, past_year_dispensations[["codigo_pac_eleito","Disp_Ultimos_12m"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl = pd.merge(Disp_semdupl, valid_dispensations[["codigo_pac_eleito","EmPrEP_Atual"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl["Disp_Ultimos_12m"] = Disp_semdupl["Disp_Ultimos_12m"].fillna("Não teve dispensação nos últimos 12 meses")
            
            # Salva os dados do último ano (mesmo parcial)
            past_year_dispensations[f"Disp_12m_{year}"] = f'Teve dispensação em {year}'
            valid_dispensations[f"EmPrEP_{year}"] = f"Em PrEP {year}"
            Disp_semdupl = pd.merge(Disp_semdupl, past_year_dispensations[["codigo_pac_eleito",f"Disp_12m_{year}"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl = pd.merge(Disp_semdupl, valid_dispensations[["codigo_pac_eleito",f"EmPrEP_{year}"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl[f"Disp_12m_{year}"] = Disp_semdupl[f"Disp_12m_{year}"].fillna(f"Não teve dispensação em {year}")
            
            # Count the unique 'codigo_pac_eleito' values in the valid dispensations to get the number of users on PrEP
            num_users_on_PrEP = valid_dispensations['codigo_pac_eleito'].nunique()
            
            # Count the number of discontinued users
            num_discontinued = past_year_dispensations.shape[0] - num_users_on_PrEP
            
            # Add new row to EmPrEP_monthly_sample
            new_row = pd.DataFrame({'Year': [year], 'Month': [month], 'Em PrEP': [num_users_on_PrEP], 'Descontinuados': [num_discontinued]})
            EmPrEP_monthly_sample = pd.concat([EmPrEP_monthly_sample, new_row], ignore_index=True)
            break        
        
        # Salva o resultado da situação Em PrEP e se teve dispensa no ano no final de cada ano para cada pessoa da base
        if month == 12:
            past_year_dispensations[f"Disp_12m_{year}"] = f'Teve dispensação em {year}'
            valid_dispensations[f"EmPrEP_{year}"] = f"Em PrEP {year}"
            Disp_semdupl = pd.merge(Disp_semdupl, past_year_dispensations[["codigo_pac_eleito",f"Disp_12m_{year}"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl = pd.merge(Disp_semdupl, valid_dispensations[["codigo_pac_eleito",f"EmPrEP_{year}"]], on="codigo_pac_eleito", how="left").copy()
            Disp_semdupl[f"Disp_12m_{year}"] = Disp_semdupl[f"Disp_12m_{year}"].fillna(f"Não teve dispensação em {year}")

        # Count the unique 'codigo_pac_eleito' values in the valid dispensations to get the number of users on PrEP
        num_users_on_PrEP = valid_dispensations['codigo_pac_eleito'].nunique()
        
        # Count the number of discontinued users
        num_discontinued = past_year_dispensations.shape[0] - num_users_on_PrEP

        # Add new row to EmPrEP_monthly_sample
        new_row = pd.DataFrame({'Year': [year], 'Month': [month], 'Em PrEP': [num_users_on_PrEP], 'Descontinuados': [num_discontinued]})
        EmPrEP_monthly_sample = pd.concat([EmPrEP_monthly_sample, new_row], ignore_index=True)
        

In [29]:
# Ajusta a coluna Em PrEP Atual e anos anteriores
Cond = [Disp_semdupl["EmPrEP_Atual"] == "Em PrEP atualmente",
        (Disp_semdupl["Disp_Ultimos_12m"] == "Teve dispensação nos últimos 12 meses") &
        (Disp_semdupl["EmPrEP_Atual"].isna())]
         
Cond_escolha = ["Em PrEP atualmente", "Estão descontinuados"]

Disp_semdupl["EmPrEP_Atual"] = np.select(Cond, Cond_escolha, default=None)

print(Disp_semdupl.drop_duplicates("codigo_pac_eleito")["EmPrEP_Atual"].value_counts())
print()

for ano_e in range(ano, 2017, -1):
    Cond1 = [Disp_semdupl[f"EmPrEP_{ano_e}"] == f"Em PrEP {ano_e}",
             (Disp_semdupl[f"Disp_12m_{ano_e}"] == f"Teve dispensação em {ano_e}") &
             (Disp_semdupl[f"EmPrEP_{ano_e}"].isna())]
         
    Cond_escolha1 = [f"Em PrEP {ano_e}", f"Descontinuou em {ano_e}"]

    Disp_semdupl[f"EmPrEP_{ano_e}"] = np.select(Cond1, Cond_escolha1, default=None)

    print(pd.DataFrame(Disp_semdupl.drop_duplicates("codigo_pac_eleito")[f"EmPrEP_{ano_e}"].value_counts()))
    print()

EmPrEP_Atual
Em PrEP atualmente      141891
Estão descontinuados     77523
Name: count, dtype: int64

                       count
EmPrEP_2025                 
Em PrEP 2025          141891
Descontinuou em 2025   77523

                       count
EmPrEP_2024                 
Em PrEP 2024          111133
Descontinuou em 2024   54549

                      count
EmPrEP_2023                
Em PrEP 2023          77455
Descontinuou em 2023  33230

                      count
EmPrEP_2022                
Em PrEP 2022          50760
Descontinuou em 2022  19592

                      count
EmPrEP_2021                
Em PrEP 2021          32780
Descontinuou em 2021  11068

                      count
EmPrEP_2020                
Em PrEP 2020          19965
Descontinuou em 2020   6632

                      count
EmPrEP_2019                
Em PrEP 2019          13561
Descontinuou em 2019   5310

                      count
EmPrEP_2018                
Em PrEP 2018           6733
Descontinuou em

In [30]:
# Criando banco com entrada única por paciente (apenas a última dispensa)
Disp_ult_unico  = Disp_semdupl.drop_duplicates("codigo_pac_eleito", keep = "first").copy()

# Define as datas de hoje e início da contagem de 12 meses
start_date = pd.to_datetime(hoje) - pd.offsets.DateOffset(years=1)
Disp_ult_unico["data_inicial"] = start_date
Disp_ult_unico["Hoje"] = pd.to_datetime(hoje)

Disp_ult_unico["EmPrEP_Atual"].value_counts(dropna = False).sort_index()

EmPrEP_Atual
Em PrEP atualmente      141891
Estão descontinuados     77523
None                    103442
Name: count, dtype: int64

In [31]:
### Banco de Cadastro

Cad['dt_ult_atu'] = pd.to_datetime(Cad['dt_ult_atu'],errors='coerce').dt.normalize()

# Trabalhando as datas

Cad['dt_nasc'] = pd.to_datetime(Cad['dt_nasc'],errors='coerce')
Cad['dt_nascimento'] = Cad['dt_nasc'].dt.normalize()


Cad['dt_cadas'] = pd.to_datetime(Cad['dt_cadas'],errors='coerce')
Cad['dt_cadastro'] = Cad['dt_cadas'].dt.normalize()

Cad['dt_ult_atu'] = pd.to_datetime(Cad['dt_ult_atu'],errors='coerce')
Cad['dt_atualizacao'] = Cad['dt_ult_atu'].dt.normalize()

#### Organiza pela data de atualização e mantém apenas o último cadastro
Cad.sort_values(by = ["dt_atualizacao"], inplace = True, ascending = False)

Cad.drop_duplicates("codigo_pac_eleito", keep = "first", inplace = True)

## Unir as bases = Banco PrEP

# Merge the dataframes with 'Cad'
PrEP = pd.merge(Disp_ult_unico, Cad, on="codigo_pac_eleito", how="left", suffixes=("_disp",""))


In [32]:
PrEP['codigo_ibge_resid'] = pd.to_numeric(PrEP['codigo_ibge_resid'],errors='coerce')
PrEP["codigo_ibge_resid"] = PrEP["codigo_ibge_resid"].fillna(PrEP["cod_ibge_udm"])
PrEP['codigo_ibge_resid'] = PrEP['codigo_ibge_resid'].astype(int)

In [33]:
# Filtrar apenas as colunas relevantes e agrupar por município e ano de primeira dispensa
pri_disp_anos = PrEP.groupby(
    ['codigo_ibge_resid', 'ano_pri_disp']
)['codigo_pac_eleito'].nunique().reset_index(name='count')

# Pivotar os dados para ter uma coluna por ano
pri_disp_pivot = pri_disp_anos.pivot_table(
    index='codigo_ibge_resid',
    columns='ano_pri_disp',
    values='count',
    fill_value=0
)

# Renomear colunas para 'Pri_disp_YYYY'
pri_disp_pivot.columns = [f'Pri_disp_{col}' for col in pri_disp_pivot.columns]

# Resetar índice para mesclar com EmPrEP_Mun
pri_disp_pivot.reset_index(inplace=True)


In [34]:
PrEP_Atual = PrEP[PrEP["EmPrEP_Atual"] == "Em PrEP atualmente"].copy()

PrEP_Atual['codigo_ibge_resid'] = pd.to_numeric(PrEP_Atual['codigo_ibge_resid'],errors='coerce')
PrEP_Atual["codigo_ibge_resid"] = PrEP_Atual["codigo_ibge_resid"].fillna(PrEP_Atual["cod_ibge_udm"])


# Criando UF e região

PrEP_Atual['Cod_UF'] = PrEP_Atual["codigo_ibge_resid"].apply(lambda x:str(x)[:2])

CondUF = [(PrEP_Atual['Cod_UF'] == '11'),
          (PrEP_Atual["Cod_UF"] == '12'),
          (PrEP_Atual["Cod_UF"] == '13'),
          (PrEP_Atual["Cod_UF"] == '14'),
          (PrEP_Atual["Cod_UF"] == '15'),
          (PrEP_Atual["Cod_UF"] == '16'),
          (PrEP_Atual["Cod_UF"] == '17'),
          (PrEP_Atual["Cod_UF"] == '21'),
          (PrEP_Atual["Cod_UF"] == '22'),
          (PrEP_Atual["Cod_UF"] == '23'),
          (PrEP_Atual["Cod_UF"] == '24'),
          (PrEP_Atual["Cod_UF"] == '25'),
          (PrEP_Atual["Cod_UF"] == '26'),
          (PrEP_Atual["Cod_UF"] == '27'),
          (PrEP_Atual["Cod_UF"] == '28'),
          (PrEP_Atual["Cod_UF"] == '29'),
          (PrEP_Atual["Cod_UF"] == '31'),
          (PrEP_Atual["Cod_UF"] == '32'),
          (PrEP_Atual["Cod_UF"] == '33'),
          (PrEP_Atual["Cod_UF"] == '35'),
          (PrEP_Atual["Cod_UF"] == '41'),
          (PrEP_Atual["Cod_UF"] == '42'),
          (PrEP_Atual["Cod_UF"] == '43'),
          (PrEP_Atual["Cod_UF"] == '50'),
          (PrEP_Atual["Cod_UF"] == '51'),
          (PrEP_Atual["Cod_UF"] == '52'),
          (PrEP_Atual["Cod_UF"] == '53')
         ]
          
UF_nomes = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']
PrEP_Atual["UF_Res"] = np.select(CondUF, UF_nomes, default="Error")

CondReg =  [(PrEP_Atual['Cod_UF'] == '11') | (PrEP_Atual["Cod_UF"] == '12') | (PrEP_Atual["Cod_UF"] == '13') | (PrEP_Atual["Cod_UF"] == '14') | (PrEP_Atual["Cod_UF"] == '15') | (PrEP_Atual["Cod_UF"] == '16') | (PrEP_Atual["Cod_UF"] == '17'),
            (PrEP_Atual["Cod_UF"] == '21') | (PrEP_Atual["Cod_UF"] == '22') | (PrEP_Atual["Cod_UF"] == '23') | (PrEP_Atual["Cod_UF"] == '24') | (PrEP_Atual["Cod_UF"] == '25') | (PrEP_Atual["Cod_UF"] == '26') | (PrEP_Atual["Cod_UF"] == '27') | (PrEP_Atual["Cod_UF"] == '28') | (PrEP_Atual["Cod_UF"] == '29'),
            (PrEP_Atual["Cod_UF"] == '31') | (PrEP_Atual["Cod_UF"] == '32') | (PrEP_Atual["Cod_UF"] == '33') | (PrEP_Atual["Cod_UF"] == '35'),
            (PrEP_Atual["Cod_UF"] == '41') | (PrEP_Atual["Cod_UF"] == '42') | (PrEP_Atual["Cod_UF"] == '43'),
            (PrEP_Atual["Cod_UF"] == '50') | (PrEP_Atual["Cod_UF"] == '51') | (PrEP_Atual["Cod_UF"] == '52') | (PrEP_Atual["Cod_UF"] == '53')
           ]
            
Regiao_nomes = ['Norte','Nordeste','Sudeste','Sul','Centro-Oeste']
PrEP_Atual["regiao_Res"] = np.select(CondReg, Regiao_nomes, default="Error")
PrEP_Atual['ano_pri_disp'] = PrEP_Atual['ano_pri_disp'].astype(int)

EmPrEP_Mun = pd.DataFrame(PrEP_Atual[['regiao_Res',"UF_Res","codigo_ibge_resid"]].value_counts(dropna = False).sort_index())

EmPrEP_Mun.rename(columns = {"count":"Em PrEP"}, inplace = True)
EmPrEP_Mun.reset_index(inplace = True)

In [35]:
PrEP2 = EmPrEP_Mun.copy()

#### Criando base para série histórica

In [36]:
PrEP_historico = pd.merge(Disp_semdupl,Cad, on="codigo_pac_eleito", how="left", suffixes=("_disp",""))

# Criando capital, UF e região

PrEP_historico['codigo_ibge_resid'] = PrEP_historico['codigo_ibge_resid'].fillna(PrEP_historico['cod_ibge_udm'])
PrEP_historico['codigo_ibge_resid'] = pd.to_numeric(PrEP_historico['codigo_ibge_resid'], errors='coerce').astype('Int64')



Cond_cap = [
    (PrEP_historico["codigo_ibge_resid"] == 1100205),
    (PrEP_historico["codigo_ibge_resid"] == 1302603),
    (PrEP_historico["codigo_ibge_resid"] == 1200401),
    (PrEP_historico["codigo_ibge_resid"] == 5002704),
    (PrEP_historico["codigo_ibge_resid"] == 1600303),
    (PrEP_historico["codigo_ibge_resid"] == 5300108),
    (PrEP_historico["codigo_ibge_resid"] == 1400100),
    (PrEP_historico["codigo_ibge_resid"] == 5103403),
    (PrEP_historico["codigo_ibge_resid"] == 1721000),
    (PrEP_historico["codigo_ibge_resid"] == 3550308),
    (PrEP_historico["codigo_ibge_resid"] == 2211001),
    (PrEP_historico["codigo_ibge_resid"] == 3304557),
    (PrEP_historico["codigo_ibge_resid"] == 1501402),
    (PrEP_historico["codigo_ibge_resid"] == 5208707),
    (PrEP_historico["codigo_ibge_resid"] == 2927408),
    (PrEP_historico["codigo_ibge_resid"] == 4205407),
    (PrEP_historico["codigo_ibge_resid"] == 2111300),
    (PrEP_historico["codigo_ibge_resid"] == 2704302),
    (PrEP_historico["codigo_ibge_resid"] == 4314902),
    (PrEP_historico["codigo_ibge_resid"] == 4106902),
    (PrEP_historico["codigo_ibge_resid"] == 3106200),
    (PrEP_historico["codigo_ibge_resid"] == 2304400),
    (PrEP_historico["codigo_ibge_resid"] == 2611606),
    (PrEP_historico["codigo_ibge_resid"] == 2507507),
    (PrEP_historico["codigo_ibge_resid"] == 2800308),
    (PrEP_historico["codigo_ibge_resid"] == 2408102),
    (PrEP_historico["codigo_ibge_resid"] == 3205309)]

cap_escolha = ["Porto Velho","Manaus","Rio Branco", "Campo Grande",
              "Macapá", 'Brasília', 'Boa Vista', 'Cuiabá', 'Palmas',
              'São Paulo', 'Teresina', 'Rio de Janeiro', 'Belém', 'Goiânia',
              'Salvador', 'Florianópolis', 'São Luís', 'Maceió',
              'Porto Alegre', 'Curitiba', 'Belo Horizonte', 'Fortaleza', 
              'Recife', 'João Pessoa', 'Aracaju', 'Natal', 'Vitória']

PrEP_historico["capital_res"] = np.select(Cond_cap, cap_escolha, default = None)

PrEP_historico['Cod_UF'] = PrEP_historico["codigo_ibge_resid"].apply(lambda x:str(x)[:2])
CondUF = [(PrEP_historico['Cod_UF'] == '11'),
          (PrEP_historico["Cod_UF"] == '12'),
          (PrEP_historico["Cod_UF"] == '13'),
          (PrEP_historico["Cod_UF"] == '14'),
          (PrEP_historico["Cod_UF"] == '15'),
          (PrEP_historico["Cod_UF"] == '16'),
          (PrEP_historico["Cod_UF"] == '17'),
          (PrEP_historico["Cod_UF"] == '21'),
          (PrEP_historico["Cod_UF"] == '22'),
          (PrEP_historico["Cod_UF"] == '23'),
          (PrEP_historico["Cod_UF"] == '24'),
          (PrEP_historico["Cod_UF"] == '25'),
          (PrEP_historico["Cod_UF"] == '26'),
          (PrEP_historico["Cod_UF"] == '27'),
          (PrEP_historico["Cod_UF"] == '28'),
          (PrEP_historico["Cod_UF"] == '29'),
          (PrEP_historico["Cod_UF"] == '31'),
          (PrEP_historico["Cod_UF"] == '32'),
          (PrEP_historico["Cod_UF"] == '33'),
          (PrEP_historico["Cod_UF"] == '35'),
          (PrEP_historico["Cod_UF"] == '41'),
          (PrEP_historico["Cod_UF"] == '42'),
          (PrEP_historico["Cod_UF"] == '43'),
          (PrEP_historico["Cod_UF"] == '50'),
          (PrEP_historico["Cod_UF"] == '51'),
          (PrEP_historico["Cod_UF"] == '52'),
          (PrEP_historico["Cod_UF"] == '53')
         ]
          
UF_nomes = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']
PrEP_historico["UF_Res"] = np.select(CondUF, UF_nomes, default="Error")


CondReg =  [(PrEP_historico['Cod_UF'] == '11') | (PrEP_historico["Cod_UF"] == '12') | (PrEP_historico["Cod_UF"] == '13') | (PrEP_historico["Cod_UF"] == '14') | (PrEP_historico["Cod_UF"] == '15') | (PrEP_historico["Cod_UF"] == '16') | (PrEP_historico["Cod_UF"] == '17'),
            (PrEP_historico["Cod_UF"] == '21') | (PrEP_historico["Cod_UF"] == '22') | (PrEP_historico["Cod_UF"] == '23') | (PrEP_historico["Cod_UF"] == '24') | (PrEP_historico["Cod_UF"] == '25') | (PrEP_historico["Cod_UF"] == '26') | (PrEP_historico["Cod_UF"] == '27') | (PrEP_historico["Cod_UF"] == '28') | (PrEP_historico["Cod_UF"] == '29'),
            (PrEP_historico["Cod_UF"] == '31') | (PrEP_historico["Cod_UF"] == '32') | (PrEP_historico["Cod_UF"] == '33') | (PrEP_historico["Cod_UF"] == '35'),
            (PrEP_historico["Cod_UF"] == '41') | (PrEP_historico["Cod_UF"] == '42') | (PrEP_historico["Cod_UF"] == '43'),
            (PrEP_historico["Cod_UF"] == '50') | (PrEP_historico["Cod_UF"] == '51') | (PrEP_historico["Cod_UF"] == '52') | (PrEP_historico["Cod_UF"] == '53')
           ]
            
Regiao_nomes = ['Norte','Nordeste','Sudeste','Sul','Centro-Oeste']
PrEP_historico["regiao_Res"] = np.select(CondReg, Regiao_nomes, default="Error")

In [37]:
PrEP_historico_mun = pd.merge(Disp_semdupl,Cad, on="codigo_pac_eleito", how="left", suffixes=("_disp",""))

# Criando capital, UF e região

PrEP_historico_mun['codigo_ibge_resid'] = PrEP_historico_mun['codigo_ibge_resid'].fillna(PrEP_historico_mun['cod_ibge_udm'])
PrEP_historico_mun['codigo_ibge_resid'] = pd.to_numeric(PrEP_historico_mun['codigo_ibge_resid'], errors='coerce').astype('Int64')

In [38]:
# Garantir que o tipo da chave seja igual nos 2 dataframes
IBGE2['codigo_ibge_resid'] = pd.to_numeric(IBGE2['codigo_ibge_resid'], errors='coerce').astype('Int64')

# Merge para trazer nome_mun
PrEP_historico_mun = PrEP_historico_mun.merge(
    IBGE2[['codigo_ibge_resid', 'nome_mun']],
    on='codigo_ibge_resid',
    how='left'
)


In [69]:
EmPrEP_Serie_Historica = pd.DataFrame([])

for ano_seq in range(2022, ano + 1):
    for mes_seq in range(1, 13):

        # Define o dia de análise e exatamente um ano antes.
        last_day = pd.Timestamp(year=ano_seq, month=mes_seq, day=1) + pd.offsets.MonthEnd(0)
        start_date = last_day - pd.offsets.DateOffset(years=1)  # um ano antes
        
        # Considera apenas as pessoas que tiveram dispensação nos 12 meses anteriores e estão EM PREP
        prep_periodo = PrEP_historico[(PrEP_historico['dt_disp'] > start_date) &
                                      (PrEP_historico['dt_disp'] <= last_day)].copy()

        prep_periodo.sort_values(by = "dt_disp", ascending = False, inplace = True)
        prep_periodo = prep_periodo.drop_duplicates(['codigo_pac_eleito'], keep = "first").copy()        

        EmPrEP_periodo = prep_periodo[prep_periodo['valid_until'] >= last_day].copy()        


        # Capital (aplica filtro para excluir Nones)
        for cap in EmPrEP_periodo['capital_res'].dropna().unique():
            EmPrEP_Serie_Historica.at[cap, f"{mes_seq}_{ano_seq}"] = EmPrEP_periodo[EmPrEP_periodo['capital_res'] == cap]["codigo_pac_eleito"].nunique()

        # Adiciona uma nova linha para cada UF ao DataFrame EmPrEP_Serie_Historica
        for uf in EmPrEP_periodo['UF_Res'].unique():
            EmPrEP_Serie_Historica.at[uf, f"{mes_seq}_{ano_seq}"] = EmPrEP_periodo[EmPrEP_periodo['UF_Res'] == uf]["codigo_pac_eleito"].nunique()      

        # Adiciona uma nova linha para cada Região ao DataFrame EmPrEP_Serie_Historica
        for regiao in EmPrEP_periodo['regiao_Res'].unique():
            EmPrEP_Serie_Historica.at[regiao, f"{mes_seq}_{ano_seq}"] = EmPrEP_periodo[EmPrEP_periodo['regiao_Res'] == regiao]["codigo_pac_eleito"].nunique()                  
                                                                                                        
        EmPrEP_Serie_Historica.at["Brasil",f"{mes_seq}_{ano_seq}"] = EmPrEP_periodo["codigo_pac_eleito"].nunique()
        
        if (ano_seq == ano) and (mes_seq == mes):
            break

        
EmPrEP_Serie_Historica

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
Rio de Janeiro,2131.0,2290.0,2514.0,2702.0,2946.0,3119.0,3302.0,3582.0,3748.0,3919.0,4004.0,4101.0,4384.0,4574.0,4811.0,4946.0,5185.0,5384.0,5593.0,5685.0,5830.0,6057.0,6096.0,6293.0,6636.0,6741.0,6882.0,7159.0,7334.0,7572.0,8151.0,8412.0,8677.0,9129.0,9028.0,9162.0,9464.0,9927.0,9907.0,9977.0,10186.0,10456.0,10791.0,11034.0,11280.0,11420.0,11229.0,11716.0
São Paulo,9850.0,10305.0,10614.0,10772.0,11129.0,11509.0,11832.0,12242.0,12389.0,12725.0,12865.0,12961.0,13585.0,14104.0,14560.0,14794.0,15291.0,15914.0,16254.0,16762.0,17054.0,17735.0,18031.0,18451.0,19175.0,19870.0,20163.0,20986.0,21244.0,22040.0,22727.0,23375.0,24297.0,24922.0,24876.0,25186.0,25912.0,27002.0,27238.0,27746.0,28018.0,28648.0,29207.0,29852.0,30224.0,30380.0,29882.0,29875.0
Florianópolis,999.0,1059.0,1102.0,1127.0,1205.0,1236.0,1295.0,1369.0,1429.0,1478.0,1518.0,1506.0,1546.0,1606.0,1642.0,1687.0,1717.0,1705.0,1715.0,1760.0,1755.0,1789.0,1819.0,1821.0,1848.0,2030.0,2072.0,2121.0,2144.0,2162.0,2243.0,2276.0,2295.0,2364.0,2349.0,2403.0,2428.0,2492.0,2498.0,2468.0,2470.0,2499.0,2540.0,2487.0,2427.0,2384.0,2402.0,2444.0
Goiânia,800.0,857.0,905.0,912.0,954.0,992.0,1027.0,1081.0,1114.0,1128.0,1159.0,1169.0,1216.0,1270.0,1321.0,1361.0,1404.0,1409.0,1432.0,1479.0,1497.0,1543.0,1582.0,1560.0,1584.0,1673.0,1696.0,1768.0,1796.0,1804.0,1837.0,1887.0,1951.0,2030.0,2050.0,2043.0,2006.0,2085.0,2054.0,2075.0,2091.0,2098.0,2097.0,2112.0,2195.0,2220.0,2200.0,2219.0
Belo Horizonte,635.0,685.0,737.0,781.0,850.0,898.0,943.0,982.0,1000.0,1028.0,1068.0,1094.0,1139.0,1170.0,1191.0,1203.0,1262.0,1326.0,1399.0,1483.0,1557.0,1624.0,1689.0,1774.0,1858.0,1843.0,1889.0,1954.0,1984.0,2027.0,2109.0,2163.0,2256.0,2446.0,2521.0,2659.0,2810.0,3076.0,3201.0,3239.0,3290.0,3455.0,3556.0,3636.0,3708.0,3788.0,3833.0,3848.0
Salvador,617.0,636.0,654.0,664.0,693.0,702.0,716.0,731.0,758.0,805.0,832.0,864.0,906.0,914.0,953.0,1001.0,1067.0,1092.0,1131.0,1152.0,1189.0,1287.0,1317.0,1402.0,1510.0,1554.0,1648.0,1709.0,1768.0,1780.0,1835.0,1918.0,2087.0,2193.0,2221.0,2312.0,2393.0,2525.0,2544.0,2599.0,2677.0,2747.0,2834.0,2870.0,2968.0,3015.0,2978.0,3012.0
Curitiba,1019.0,1089.0,1126.0,1171.0,1214.0,1222.0,1287.0,1334.0,1375.0,1365.0,1377.0,1346.0,1408.0,1443.0,1537.0,1545.0,1619.0,1679.0,1712.0,1742.0,1758.0,1846.0,1869.0,1863.0,1904.0,1983.0,2014.0,2100.0,2108.0,2190.0,2280.0,2355.0,2427.0,2486.0,2556.0,2562.0,2585.0,2656.0,2675.0,2707.0,2672.0,2730.0,2791.0,2838.0,2871.0,2913.0,2889.0,2907.0
Porto Alegre,790.0,829.0,893.0,928.0,996.0,1031.0,1052.0,1111.0,1117.0,1170.0,1200.0,1267.0,1285.0,1344.0,1391.0,1447.0,1491.0,1575.0,1615.0,1658.0,1659.0,1729.0,1794.0,1875.0,1912.0,1978.0,2017.0,2062.0,1925.0,1765.0,1848.0,1936.0,2048.0,2184.0,2216.0,2280.0,2381.0,2429.0,2440.0,2457.0,2467.0,2538.0,2606.0,2581.0,2633.0,2746.0,2829.0,2893.0
Campo Grande,391.0,408.0,450.0,434.0,440.0,483.0,522.0,546.0,570.0,596.0,605.0,616.0,647.0,648.0,640.0,588.0,583.0,622.0,702.0,689.0,705.0,710.0,731.0,740.0,762.0,773.0,778.0,810.0,807.0,846.0,886.0,900.0,878.0,897.0,918.0,926.0,924.0,970.0,952.0,947.0,949.0,964.0,1004.0,1001.0,1027.0,1036.0,1046.0,1072.0
Manaus,649.0,692.0,714.0,694.0,737.0,742.0,766.0,814.0,799.0,827.0,841.0,864.0,878.0,907.0,925.0,931.0,985.0,990.0,1007.0,1057.0,1042.0,1102.0,1138.0,1184.0,1299.0,1318.0,1321.0,1369.0,1411.0,1462.0,1524.0,1580.0,1603.0,1612.0,1630.0,1670.0,1684.0,1780.0,1786.0,1821.0,1832.0,1890.0,1965.0,2021.0,2076.0,2111.0,2101.0,2095.0


## Cálculo do Indicador Atual por Município

In [40]:
EmPrEP_Serie_Historica_mun = pd.DataFrame([])

for ano_seq in range(2022, ano + 1):
    for mes_seq in range(1, 13):

        # Define o dia de análise e exatamente um ano antes
        last_day = pd.Timestamp(year=ano_seq, month=mes_seq, day=1) + pd.offsets.MonthEnd(0)
        start_date = last_day - pd.offsets.DateOffset(years=1)

        # Pessoas com dispensação nos 12 meses anteriores e em PrEP
        prep_periodo = PrEP_historico_mun[
            (PrEP_historico_mun['dt_disp'] > start_date) &
            (PrEP_historico_mun['dt_disp'] <= last_day)
        ].copy()

        prep_periodo.sort_values(by="dt_disp", ascending=False, inplace=True)
        prep_periodo = prep_periodo.drop_duplicates(['codigo_pac_eleito'], keep="first").copy()

        EmPrEP_periodo = prep_periodo[prep_periodo['valid_until'] >= last_day].copy()

        # agrupa por codigo_ibge_resid e nome_mun
        mun_periodo = (
            EmPrEP_periodo
            .dropna(subset=['codigo_ibge_resid', 'nome_mun'])
            .groupby(['codigo_ibge_resid', 'nome_mun'])['codigo_pac_eleito']
            .nunique()
            .reset_index()
        )

        # preenche a série histórica por municipio
        for _, row in mun_periodo.iterrows():
            cod = row['codigo_ibge_resid']
            nome = row['nome_mun']
            qtde = row['codigo_pac_eleito']

            # garante que o nome do municipio fique em uma coluna fixa
            EmPrEP_Serie_Historica_mun.at[cod, 'nome_mun'] = nome
            EmPrEP_Serie_Historica_mun.at[cod, f"{mes_seq}_{ano_seq}"] = qtde

    
        if (ano_seq == ano) and (mes_seq == mes):
            break
EmPrEP_Serie_Historica_mun

,nome_mun,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
1100023,Ariquemes,3.0,3.0,4.0,3.0,3.0,2.0,2.0,4.0,5.0,4.0,5.0,5.0,6.0,6.0,4.0,5.0,7.0,11.0,11.0,10.0,11.0,14.0,15.0,13.0,15.0,14.0,15.0,13.0,6.0,9.0,8.0,11.0,9.0,14.0,13.0,12.0,9.0,12.0,11.0,10.0,12.0,15.0,19.0,22.0,20.0,20.0,17.0,15.0
1100049,Cacoal,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,5.0,4.0,5.0,5.0,6.0,5.0,9.0,11.0,9.0,8.0,8.0,12.0,13.0,11.0,14.0,14.0,21.0,23.0,23.0,23.0,23.0,23.0,24.0,24.0,25.0,22.0,13.0,19.0,23.0,25.0,29.0,31.0,38.0,38.0,35.0,32.0,34.0
1100098,Espigão D'Oeste,1.0,1.0,1.0,1.0,1.0,NaN,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,5.0
1100106,Guajará-Mirim,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,3.0,6.0,6.0,4.0,5.0,8.0,6.0,6.0,3.0,4.0,5.0,7.0,9.0,9.0,8.0,8.0,10.0,10.0,11.0,12.0,12.0,9.0,9.0,11.0,13.0,9.0,9.0,8.0,10.0,11.0,9.0,9.0,11.0,11.0,10.0,7.0
1100114,Jaru,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,5.0,6.0,5.0,4.0,7.0,9.0,7.0,6.0,8.0,5.0,4.0,5.0,6.0,5.0,5.0,6.0,9.0,10.0,10.0,9.0,10.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102793,Carlinda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5107065,Querência,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5204102,Cachoeira Alta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5204904,Campos Belos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [41]:
PrEP2['codigo_ibge_resid'] = PrEP2['codigo_ibge_resid'].astype(int)

In [42]:
PrEP2["codigo_ibge_resid"] = PrEP2["codigo_ibge_resid"].astype("string").str.zfill(7)
IBGE2["codigo_ibge_resid"] = IBGE2["codigo_ibge_resid"].astype("string").str.zfill(7)

Indicador = IBGE2.merge(
    PrEP2[['codigo_ibge_resid', 'Em PrEP']],
    on='codigo_ibge_resid',
    how='left'
)

Indicador.rename(columns={
    'nome_mun': 'Município',
    'Região': 'regiao_Res',
    'UF': 'UF_Res'
}, inplace=True)

# Preencher os NaN com 0 e remover casas decimais
Indicador['Em PrEP'] = Indicador['Em PrEP'].fillna(0).astype(int)

Indicador

,regiao_Res,UF_Res,Nome_UF,codigo_ibge_resid,Município,Vinculados,Em PrEP
0,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,3,1
1,Norte,11,Rondônia,1100379,Alto Alegre dos Parecis,0,0
2,Norte,11,Rondônia,1100403,Alto Paraíso,1,0
3,Norte,11,Rondônia,1100346,Alvorada D'Oeste,0,0
4,Norte,11,Rondônia,1100023,Ariquemes,18,15
...,...,...,...,...,...,...,...
5565,Centro-Oeste,52,Goiás,5222005,Vianópolis,0,2
5566,Centro-Oeste,52,Goiás,5222054,Vicentinópolis,0,0
5567,Centro-Oeste,52,Goiás,5222203,Vila Boa,0,0
5568,Centro-Oeste,52,Goiás,5222302,Vila Propício,0,0


In [43]:
Pop_ibge["codigo_ibge_resid"] = Pop_ibge["codigo_ibge_resid"].astype(str)
Indicador["codigo_ibge_resid"] = Indicador["codigo_ibge_resid"].astype(str)

Indicador = pd.merge(
    Indicador,
    Pop_ibge[["Populacao_resid", "codigo_ibge_resid"]],
    on="codigo_ibge_resid",
    how="left"
)
Indicador['Populacao_resid'] = Indicador['Populacao_resid'].astype('Int64')
Indicador

,regiao_Res,UF_Res,Nome_UF,codigo_ibge_resid,Município,Vinculados,Em PrEP,Populacao_resid
0,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,3,1,21495
1,Norte,11,Rondônia,1100379,Alto Alegre dos Parecis,0,0,11479
2,Norte,11,Rondônia,1100403,Alto Paraíso,1,0,16320
3,Norte,11,Rondônia,1100346,Alvorada D'Oeste,0,0,13117
4,Norte,11,Rondônia,1100023,Ariquemes,18,15,96833
...,...,...,...,...,...,...,...,...
5565,Centro-Oeste,52,Goiás,5222005,Vianópolis,0,2,14956
5566,Centro-Oeste,52,Goiás,5222054,Vicentinópolis,0,0,8768
5567,Centro-Oeste,52,Goiás,5222203,Vila Boa,0,0,4215
5568,Centro-Oeste,52,Goiás,5222302,Vila Propício,0,0,5815


In [44]:
Indicador["Indicador_Mun"] = round(Indicador["Em PrEP"]/Indicador["Vinculados"],2)

In [45]:
# Aplicar as regras para criação da coluna 'Grupo'
def classificar_grupo(row):
    if row['Vinculados'] == 0 and row['Em PrEP'] == 0:
        return 'Sem novos vinculados e sem PrEP'
    elif row['Vinculados'] == 0 and row['Em PrEP'] >= 1:
        return 'Sem novos vinculados, com pessoas em PrEP'
    elif row['Indicador_Mun'] < 1:
        return 'Grupo 0'
    elif row['Indicador_Mun'] < 2:
        return 'Grupo 1'
    elif row['Indicador_Mun'] < 3:
        return 'Grupo 2'
    elif row['Indicador_Mun'] < 4:
        return 'Grupo 3'
    else:
        return 'Grupo 4'

# Aplicar ao DataFrame
Indicador['Grupo'] = Indicador.apply(classificar_grupo, axis=1)

Indicador['Grupo'].value_counts()


Grupo
Sem novos vinculados e sem PrEP              2042
Grupo 0                                      1391
Sem novos vinculados, com pessoas em PrEP     805
Grupo 1                                       584
Grupo 4                                       348
Grupo 2                                       260
Grupo 3                                       140
Name: count, dtype: int64

In [46]:
# Contando o número de municípios em cada grupo
contagem_grupos = Indicador['Grupo'].value_counts()
ordem_desejada = ["Sem novos vinculados e sem PrEP",'Sem novos vinculados, com pessoas em PrEP', "Grupo 0", "Grupo 1", "Grupo 2", "Grupo 3", "Grupo 4"]

# Calculando a porcentagem de cada grupo em relação ao total
porcentagem_grupos = (contagem_grupos / contagem_grupos.sum()) * 100
porcentagem_grupos_arredondada = porcentagem_grupos.round(1)

# Combinando contagem e porcentagem arredondada em um DataFrame
tabela_resumo_arredondada = pd.DataFrame({'Qtd municípios': contagem_grupos, '%': porcentagem_grupos_arredondada})

# Reordenando a tabela resumo com base na ordem desejada
tabela_resumo_ordenada = tabela_resumo_arredondada.reindex(ordem_desejada)

# Calculando o total de municípios
total_municipios = tabela_resumo_arredondada['Qtd municípios'].sum()

# Criando uma nova linha com o total
linha_total = pd.DataFrame({'Qtd municípios': [total_municipios], '%': [100]}, index=['Total'])

# Usando concat para anexar a linha de total ao final da tabela resumo ordenada
tabela_geral_mun = pd.concat([tabela_resumo_ordenada, linha_total])
tabela_geral_mun


,Qtd municípios,%
Sem novos vinculados e sem PrEP,2042,36.7
"Sem novos vinculados, com pessoas em PrEP",805,14.5
Grupo 0,1391,25.0
Grupo 1,584,10.5
Grupo 2,260,4.7
Grupo 3,140,2.5
Grupo 4,348,6.2
Total,5570,100.0


In [47]:
# Contando o número de municípios maiores de 50k habitantes em cada grupo
contagem_grupos = Indicador[Indicador["Populacao_resid"] >= 50000]['Grupo'].value_counts()
ordem_desejada = ["Sem novos vinculados e sem PrEP",'Sem novos vinculados, com pessoas em PrEP', "Grupo 0", "Grupo 1", "Grupo 2", "Grupo 3", "Grupo 4"]

# Calculando a porcentagem de cada grupo em relação ao total
porcentagem_grupos = (contagem_grupos / contagem_grupos.sum()) * 100
porcentagem_grupos_arredondada = porcentagem_grupos.round(1)

# Combinando contagem e porcentagem arredondada em um DataFrame
tabela_resumo_arredondada = pd.DataFrame({'Qtd municípios >=50.000 hab': contagem_grupos, '%': porcentagem_grupos_arredondada})

# Reordenando a tabela resumo com base na ordem desejada
tabela_resumo_ordenada = tabela_resumo_arredondada.reindex(ordem_desejada)

# Calculando o total de municípios
total_municipios = int(tabela_resumo_arredondada['Qtd municípios >=50.000 hab'].sum())

# Criando uma nova linha com o total
linha_total = pd.DataFrame({'Qtd municípios >=50.000 hab': [total_municipios], '%': [100]}, index=['Total'])

# Usando concat para anexar a linha de total ao final da tabela resumo ordenada
tabela_mun_50k = pd.concat([tabela_resumo_ordenada, linha_total])
tabela_mun_50k['Qtd municípios >=50.000 hab'] = tabela_mun_50k['Qtd municípios >=50.000 hab'].astype('Int64')


tabela_mun_50k

,Qtd municípios >=50.000 hab,%
Sem novos vinculados e sem PrEP,1,0.2
"Sem novos vinculados, com pessoas em PrEP",3,0.5
Grupo 0,125,19.1
Grupo 1,125,19.1
Grupo 2,102,15.5
Grupo 3,73,11.1
Grupo 4,227,34.6
Total,656,100.0


In [48]:
Tabelao_mun = Indicador.copy()

In [49]:
# Certifique-se de que 'codigo_ibge_resid' está no tipo correto
PrEP['codigo_ibge_resid'] = PrEP['codigo_ibge_resid'].astype(str)
Tabelao_mun['codigo_ibge_resid'] = Tabelao_mun['codigo_ibge_resid'].astype(str)

# Lista dos anos disponíveis
anos = list(range(2018, 2026))  # 2026 exclusivo

# Loop para calcular e adicionar as colunas de EmPrEP por ano
for ano in anos:
    col = f'EmPrEP_{ano}'
    
    if col in PrEP.columns:
        # Filtra apenas quem está Em PrEP no final do ano
        prep_ano = PrEP[PrEP[col] == f'Em PrEP {ano}']
        
        # Conta por município
        prep_count = prep_ano.groupby('codigo_ibge_resid').size().reset_index(name=col)
        
        # Junta com o Tabelao_mun
        Tabelao_mun = Tabelao_mun.merge(prep_count, on='codigo_ibge_resid', how='left')
        
        # Preenche nulos com 0
        Tabelao_mun[col] = Tabelao_mun[col].fillna(0).astype(int)


In [50]:
pri_disp_pivot['codigo_ibge_resid'] = pri_disp_pivot['codigo_ibge_resid'].astype(str)

# Unir com EmPrEP_Mun usando 'codigo_ibge_resid'
Tabelao_mun = Tabelao_mun.merge(pri_disp_pivot, on='codigo_ibge_resid', how='left')

# Preencher possíveis NaNs com 0
Tabelao_mun.fillna(0, inplace=True)

# Garantir que as novas colunas sejam inteiras
pri_disp_cols = [col for col in Tabelao_mun.columns if col.startswith('Pri_disp_')]
Tabelao_mun[pri_disp_cols] = Tabelao_mun[pri_disp_cols].astype(int)

Tabelao_mun

,regiao_Res,UF_Res,Nome_UF,codigo_ibge_resid,Município,Vinculados,Em PrEP,Populacao_resid,Indicador_Mun,Grupo,EmPrEP_2018,EmPrEP_2019,EmPrEP_2020,EmPrEP_2021,EmPrEP_2022,EmPrEP_2023,EmPrEP_2024,EmPrEP_2025,Pri_disp_2018,Pri_disp_2019,Pri_disp_2020,Pri_disp_2021,Pri_disp_2022,Pri_disp_2023,Pri_disp_2024,Pri_disp_2025
0,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,3,1,21495,0.33,Grupo 0,0,1,0,0,0,0,2,1,0,1,0,0,0,0,2,1
1,Norte,11,Rondônia,1100379,Alto Alegre dos Parecis,0,0,11479,0.00,Sem novos vinculados e sem PrEP,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,Norte,11,Rondônia,1100403,Alto Paraíso,1,0,16320,0.00,Grupo 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Norte,11,Rondônia,1100346,Alvorada D'Oeste,0,0,13117,0.00,Sem novos vinculados e sem PrEP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Norte,11,Rondônia,1100023,Ariquemes,18,15,96833,0.83,Grupo 0,0,0,1,2,5,13,12,15,0,0,1,2,6,19,11,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,Centro-Oeste,52,Goiás,5222005,Vianópolis,0,2,14956,inf,"Sem novos vinculados, com pessoas em PrEP",0,0,0,0,1,2,2,2,0,0,0,0,1,1,0,1
5566,Centro-Oeste,52,Goiás,5222054,Vicentinópolis,0,0,8768,0.00,Sem novos vinculados e sem PrEP,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
5567,Centro-Oeste,52,Goiás,5222203,Vila Boa,0,0,4215,0.00,Sem novos vinculados e sem PrEP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5568,Centro-Oeste,52,Goiás,5222302,Vila Propício,0,0,5815,0.00,Sem novos vinculados e sem PrEP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Cálculo do Indicador Atual por UF

In [51]:
Indicador['Vinculados_UF'] = Indicador.groupby(["UF_Res"])['Vinculados'].transform('sum')
Indicador['Em PrEP_UF'] = Indicador.groupby(["UF_Res"])['Em PrEP'].transform('sum')
Indicador["Indicador_UF"] = round(Indicador["Em PrEP_UF"]/Indicador["Vinculados_UF"],2)
Indicador_UF = Indicador.drop_duplicates("UF_Res")[["UF_Res","Vinculados_UF","Em PrEP_UF","Indicador_UF"]].copy()

In [52]:
# Função para determinar o grupo
def classificar_grupo(row):
    if row['Vinculados_UF'] == 0 and row['Em PrEP_UF'] == 0:
        return 'Sem novos vinculados e sem PrEP'
    elif row['Vinculados_UF'] == 0 and row['Em PrEP_UF'] >= 1:
        return 'Sem  novos vinculados, com pessoas em PrEP'
    elif row['Indicador_UF'] < 1:
        return 'Grupo 0'
    elif row['Indicador_UF'] < 2:
        return 'Grupo 1'
    elif row['Indicador_UF'] < 3:
        return 'Grupo 2'
    elif row['Indicador_UF'] < 4:
        return 'Grupo 3'
    else:
        return 'Grupo 4'

# Criando a nova coluna 'Grupo'
# Criando a nova coluna 'Grupo'
Indicador_UF['Grupo'] = Indicador_UF.apply(classificar_grupo, axis=1)

Indicador_UF


,UF_Res,Vinculados_UF,Em PrEP_UF,Indicador_UF,Grupo
0,11,245,488,1.99,Grupo 1
52,12,104,197,1.89,Grupo 1
74,13,1138,2251,1.98,Grupo 1
136,14,173,295,1.71,Grupo 1
151,15,1821,2342,1.29,Grupo 1
295,16,215,353,1.64,Grupo 1
311,17,205,441,2.15,Grupo 2
450,21,1112,1438,1.29,Grupo 1
667,22,411,939,2.28,Grupo 2
891,23,1378,5444,3.95,Grupo 3


In [53]:
# Contando o número de municípios em cada grupo
contagem_grupos = Indicador_UF['Grupo'].value_counts()

# Ordem desejada
ordem_desejada = ["Sem novos vinculados e sem PrEP",'Sem novos vinculados, com pessoas em PrEP', "Grupo 0", "Grupo 1", "Grupo 2", "Grupo 3", "Grupo 4"]

# Calculando a porcentagem de cada grupo em relação ao total
porcentagem_grupos = (contagem_grupos / contagem_grupos.sum()) * 100
porcentagem_grupos_arredondada = porcentagem_grupos.round(1)

# Combinando contagem e porcentagem arredondada em um DataFrame
tabela_resumo_arredondada = pd.DataFrame({'Qtd UF': contagem_grupos, '%': porcentagem_grupos_arredondada})

# Reordenando a tabela resumo com base na ordem desejada
tabela_resumo_ordenada = tabela_resumo_arredondada.reindex(ordem_desejada)

# Convertendo 'Qtd UF' para int para evitar casas decimais
tabela_resumo_ordenada['Qtd UF'] = tabela_resumo_ordenada['Qtd UF'].fillna(0).astype(int)

# Calculando o total de municípios
total_UF = tabela_resumo_ordenada['Qtd UF'].sum()

# Criando uma nova linha com o total
linha_total = pd.DataFrame({'Qtd UF': [total_UF], '%': [100]}, index=['Total'])

# Usando concat para anexar a linha de total ao final da tabela resumo ordenada
tabela_geral_UF = pd.concat([tabela_resumo_ordenada, linha_total])
tabela_geral_UF


,Qtd UF,%
Sem novos vinculados e sem PrEP,0,NaN
"Sem novos vinculados, com pessoas em PrEP",0,NaN
Grupo 0,0,NaN
Grupo 1,8,29.6
Grupo 2,7,25.9
Grupo 3,6,22.2
Grupo 4,6,22.2
Total,27,100.0


## Cálculo do Indicador Atual por Região

In [54]:
Indicador['Vinculados_Reg'] = Indicador.groupby(["regiao_Res"])['Vinculados'].transform('sum')
Indicador['Em PrEP_Reg'] = Indicador.groupby(["regiao_Res"])['Em PrEP'].transform('sum')
Indicador["Indicador_Reg"] = round(Indicador["Em PrEP_Reg"]/Indicador["Vinculados_Reg"],2)
Indicador_Reg = Indicador.drop_duplicates("regiao_Res")[["regiao_Res","Vinculados_Reg","Em PrEP_Reg","Indicador_Reg"]].copy()

In [55]:
# Função para determinar o grupo
def classificar_grupo(row):
    if row['Vinculados_Reg'] == 0 and row['Em PrEP_Reg'] == 0:
        return 'Sem novos vinculados e sem PrEP'
    elif row['Vinculados_Reg'] == 0 and row['Em PrEP_Reg'] >= 1:
        return 'Sem  novos vinculados, com pessoas em PrEP'
    elif row['Indicador_Reg'] < 1:
        return 'Grupo 0'
    elif row['Indicador_Reg'] < 2:
        return 'Grupo 1'
    elif row['Indicador_Reg'] < 3:
        return 'Grupo 2'
    elif row['Indicador_Reg'] < 4:
        return 'Grupo 3'
    else:
        return 'Grupo 4'

# Criando a nova coluna 'Grupo'
Indicador_Reg['Grupo'] = Indicador_Reg.apply(classificar_grupo, axis=1)

Indicador_Reg


,regiao_Res,Vinculados_Reg,Em PrEP_Reg,Indicador_Reg,Grupo
0,Norte,3901,6367,1.63,Grupo 1
450,Nordeste,7914,21699,2.74,Grupo 2
2244,Sudeste,11161,81655,7.32,Grupo 4
3912,Sul,4760,20850,4.38,Grupo 4
5103,Centro-Oeste,2759,11320,4.10,Grupo 4


## Cálculo do Indicador Nacional Atual

In [56]:
Indicador["Nac"] = 1
Indicador['Vinculados_Nac'] = Indicador.groupby(["Nac"])['Vinculados'].transform('sum')
Indicador['Em PrEP_Nac'] = Indicador.groupby(["Nac"])['Em PrEP'].transform('sum')
Indicador["Indicador_Nac"] = round(Indicador["Em PrEP_Nac"]/Indicador["Vinculados_Nac"],2)
Indicador_Nac = Indicador.drop_duplicates("Nac")[["Vinculados_Nac","Em PrEP_Nac","Indicador_Nac"]].copy()

In [57]:
# Função para determinar o grupo
def determinar_grupo(valor):
    if valor == "Sem Novos Vinculados":
        return None  # Retorna um valor nulo para esta condição específica
    elif valor < 1:
        return "Grupo 0"
    elif 1 <= valor < 2:
        return "Grupo 1"
    elif 2 <= valor < 3:
        return "Grupo 2"
    elif 3 <= valor < 4:
        return "Grupo 3"
    else:
        return "Grupo 4"

# Criando a nova coluna 'Grupo'
Indicador_Nac['Grupo'] = Indicador_Nac['Indicador_Nac'].apply(determinar_grupo)

Indicador_Nac

,Vinculados_Nac,Em PrEP_Nac,Indicador_Nac,Grupo
0,30495,141891,4.65,Grupo 4


## Avaliação da Série Temporal do indicador por UF, Região e Brasil

In [58]:
EmPrEP_Serie_Historica

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
Rio de Janeiro,2131.0,2290.0,2514.0,2702.0,2946.0,3119.0,3302.0,3582.0,3748.0,3919.0,4004.0,4101.0,4384.0,4574.0,4811.0,4946.0,5185.0,5384.0,5593.0,5685.0,5830.0,6057.0,6096.0,6293.0,6636.0,6741.0,6882.0,7159.0,7334.0,7572.0,8151.0,8412.0,8677.0,9129.0,9028.0,9162.0,9464.0,9927.0,9907.0,9977.0,10186.0,10456.0,10791.0,11034.0,11280.0,11420.0,11229.0,11716.0
São Paulo,9850.0,10305.0,10614.0,10772.0,11129.0,11509.0,11832.0,12242.0,12389.0,12725.0,12865.0,12961.0,13585.0,14104.0,14560.0,14794.0,15291.0,15914.0,16254.0,16762.0,17054.0,17735.0,18031.0,18451.0,19175.0,19870.0,20163.0,20986.0,21244.0,22040.0,22727.0,23375.0,24297.0,24922.0,24876.0,25186.0,25912.0,27002.0,27238.0,27746.0,28018.0,28648.0,29207.0,29852.0,30224.0,30380.0,29882.0,29875.0
Florianópolis,999.0,1059.0,1102.0,1127.0,1205.0,1236.0,1295.0,1369.0,1429.0,1478.0,1518.0,1506.0,1546.0,1606.0,1642.0,1687.0,1717.0,1705.0,1715.0,1760.0,1755.0,1789.0,1819.0,1821.0,1848.0,2030.0,2072.0,2121.0,2144.0,2162.0,2243.0,2276.0,2295.0,2364.0,2349.0,2403.0,2428.0,2492.0,2498.0,2468.0,2470.0,2499.0,2540.0,2487.0,2427.0,2384.0,2402.0,2444.0
Goiânia,800.0,857.0,905.0,912.0,954.0,992.0,1027.0,1081.0,1114.0,1128.0,1159.0,1169.0,1216.0,1270.0,1321.0,1361.0,1404.0,1409.0,1432.0,1479.0,1497.0,1543.0,1582.0,1560.0,1584.0,1673.0,1696.0,1768.0,1796.0,1804.0,1837.0,1887.0,1951.0,2030.0,2050.0,2043.0,2006.0,2085.0,2054.0,2075.0,2091.0,2098.0,2097.0,2112.0,2195.0,2220.0,2200.0,2219.0
Belo Horizonte,635.0,685.0,737.0,781.0,850.0,898.0,943.0,982.0,1000.0,1028.0,1068.0,1094.0,1139.0,1170.0,1191.0,1203.0,1262.0,1326.0,1399.0,1483.0,1557.0,1624.0,1689.0,1774.0,1858.0,1843.0,1889.0,1954.0,1984.0,2027.0,2109.0,2163.0,2256.0,2446.0,2521.0,2659.0,2810.0,3076.0,3201.0,3239.0,3290.0,3455.0,3556.0,3636.0,3708.0,3788.0,3833.0,3848.0
Salvador,617.0,636.0,654.0,664.0,693.0,702.0,716.0,731.0,758.0,805.0,832.0,864.0,906.0,914.0,953.0,1001.0,1067.0,1092.0,1131.0,1152.0,1189.0,1287.0,1317.0,1402.0,1510.0,1554.0,1648.0,1709.0,1768.0,1780.0,1835.0,1918.0,2087.0,2193.0,2221.0,2312.0,2393.0,2525.0,2544.0,2599.0,2677.0,2747.0,2834.0,2870.0,2968.0,3015.0,2978.0,3012.0
Curitiba,1019.0,1089.0,1126.0,1171.0,1214.0,1222.0,1287.0,1334.0,1375.0,1365.0,1377.0,1346.0,1408.0,1443.0,1537.0,1545.0,1619.0,1679.0,1712.0,1742.0,1758.0,1846.0,1869.0,1863.0,1904.0,1983.0,2014.0,2100.0,2108.0,2190.0,2280.0,2355.0,2427.0,2486.0,2556.0,2562.0,2585.0,2656.0,2675.0,2707.0,2672.0,2730.0,2791.0,2838.0,2871.0,2913.0,2889.0,2907.0
Porto Alegre,790.0,829.0,893.0,928.0,996.0,1031.0,1052.0,1111.0,1117.0,1170.0,1200.0,1267.0,1285.0,1344.0,1391.0,1447.0,1491.0,1575.0,1615.0,1658.0,1659.0,1729.0,1794.0,1875.0,1912.0,1978.0,2017.0,2062.0,1925.0,1765.0,1848.0,1936.0,2048.0,2184.0,2216.0,2280.0,2381.0,2429.0,2440.0,2457.0,2467.0,2538.0,2606.0,2581.0,2633.0,2746.0,2829.0,2893.0
Campo Grande,391.0,408.0,450.0,434.0,440.0,483.0,522.0,546.0,570.0,596.0,605.0,616.0,647.0,648.0,640.0,588.0,583.0,622.0,702.0,689.0,705.0,710.0,731.0,740.0,762.0,773.0,778.0,810.0,807.0,846.0,886.0,900.0,878.0,897.0,918.0,926.0,924.0,970.0,952.0,947.0,949.0,964.0,1004.0,1001.0,1027.0,1036.0,1046.0,1072.0
Manaus,649.0,692.0,714.0,694.0,737.0,742.0,766.0,814.0,799.0,827.0,841.0,864.0,878.0,907.0,925.0,931.0,985.0,990.0,1007.0,1057.0,1042.0,1102.0,1138.0,1184.0,1299.0,1318.0,1321.0,1369.0,1411.0,1462.0,1524.0,1580.0,1603.0,1612.0,1630.0,1670.0,1684.0,1780.0,1786.0,1821.0,1832.0,1890.0,1965.0,2021.0,2076.0,2111.0,2101.0,2095.0


In [59]:
Vinculados

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
Manaus,1041.0,1026.0,966.0,954.0,965.0,998.0,1003.0,997.0,983.0,991.0,999.0,952.0,962.0,939.0,947.0,951.0,914.0,934.0,928.0,982.0,969.0,964.0,996.0,981.0,1007.0,1003.0,983.0,1032.0,1016.0,1025.0,1010.0,1025.0,1035.0,1032.0,1016.0,1016.0,1002.0,1032.0,1021.0,985.0,974.0,967.0,977.0,928.0,884.0,894.0,900.0,875.0
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maceió,309.0,300.0,293.0,293.0,280.0,278.0,279.0,290.0,288.0,291.0,279.0,278.0,297.0,283.0,283.0,283.0,302.0,311.0,308.0,337.0,333.0,333.0,324.0,312.0,297.0,288.0,298.0,308.0,321.0,322.0,328.0,328.0,328.0,324.0,321.0,327.0,337.0,352.0,327.0,333.0,319.0,310.0,310.0,312.0,323.0,315.0,320.0,320.0
Rio de Janeiro,1789.0,1858.0,1894.0,1907.0,1911.0,1953.0,1967.0,1977.0,1911.0,1858.0,1827.0,1844.0,1871.0,1854.0,1889.0,1882.0,1930.0,1904.0,1843.0,1838.0,1807.0,1832.0,1823.0,1819.0,1891.0,1904.0,1892.0,1900.0,1923.0,1903.0,1899.0,1929.0,1895.0,1931.0,1917.0,1934.0,1911.0,1960.0,1914.0,1836.0,1785.0,1745.0,1739.0,1682.0,1721.0,1795.0,1795.0,1799.0
Curitiba,388.0,376.0,408.0,397.0,411.0,417.0,432.0,440.0,430.0,443.0,405.0,405.0,408.0,406.0,407.0,418.0,462.0,472.0,467.0,496.0,492.0,495.0,462.0,439.0,452.0,427.0,412.0,414.0,429.0,432.0,426.0,443.0,434.0,448.0,427.0,416.0,418.0,405.0,410.0,387.0,409.0,429.0,414.0,428.0,404.0,415.0,392.0,368.0
São Luís,344.0,349.0,349.0,350.0,355.0,366.0,368.0,371.0,349.0,347.0,339.0,334.0,357.0,357.0,381.0,382.0,387.0,376.0,367.0,381.0,350.0,347.0,341.0,345.0,337.0,348.0,354.0,376.0,371.0,374.0,375.0,367.0,360.0,354.0,340.0,344.0,356.0,352.0,356.0,350.0,351.0,341.0,341.0,347.0,366.0,376.0,376.0,367.0
Porto Velho,148.0,143.0,134.0,132.0,152.0,148.0,161.0,177.0,164.0,164.0,156.0,163.0,160.0,179.0,189.0,184.0,189.0,185.0,193.0,184.0,172.0,174.0,178.0,178.0,160.0,157.0,156.0,153.0,146.0,159.0,172.0,178.0,183.0,181.0,195.0,181.0,182.0,178.0,173.0,193.0,196.0,188.0,179.0,170.0,176.0,152.0,129.0,125.0
João Pessoa,199.0,210.0,187.0,187.0,197.0,197.0,187.0,188.0,213.0,218.0,198.0,213.0,237.0,242.0,242.0,239.0,262.0,257.0,268.0,272.0,262.0,278.0,270.0,270.0,278.0,284.0,279.0,289.0,297.0,297.0,277.0,269.0,276.0,282.0,278.0,282.0,303.0,312.0,306.0,295.0,292.0,284.0,271.0,267.0,252.0,242.0,236.0,232.0
Fortaleza,745.0,744.0,740.0,723.0,720.0,686.0,661.0,676.0,701.0,709.0,722.0,752.0,808.0,801.0,757.0,748.0,742.0,718.0,695.0,724.0,726.0,751.0,737.0,735.0,734.0,731.0,689.0,677.0,667.0,694.0,716.0,691.0,773.0,778.0,755.0,781.0,738.0,762.0,685.0,709.0,722.0,745.0,714.0,722.0,765.0,731.0,739.0,632.0
Goiânia,420.0,426.0,412.0,418.0,411.0,392.0,396.0,404.0,418.0,408.0,418.0,419.0,441.0,443.0,420.0,424.0,429.0,429.0,411.0,415.0,398.0,411.0,431.0,432.0,439.0,422.0,441.0,446.0,408.0,419.0,421.0,432.0,416.0,413.0,405.0,387.0,401.0,416.0,407.0,390.0,395.0,392.0,359.0,320.0,338.0,335.0,333.0,360.0


In [60]:
Indicador_mes = round(EmPrEP_Serie_Historica/Vinculados,2).reindex(sorted(cap_escolha)+sorted(UF_nomes)+sorted(Regiao_nomes)+["Brasil"])
Indicador_mes

,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
Aracaju,0.58,0.65,0.74,0.76,0.86,0.91,0.92,0.94,0.94,1.00,0.95,0.88,0.98,1.00,1.02,1.08,1.24,1.32,1.40,1.41,1.58,1.70,1.77,1.60,1.73,1.83,1.77,1.82,1.84,2.11,2.16,2.32,2.28,2.31,2.36,2.36,2.41,2.35,2.41,2.58,2.75,2.86,2.98,3.37,3.88,4.09,4.06,4.09
Belo Horizonte,1.36,1.39,1.53,1.66,1.74,1.88,1.94,2.02,2.02,2.12,2.30,2.43,2.45,2.64,2.76,2.82,2.80,2.88,3.16,3.06,3.21,3.15,3.32,3.46,3.45,3.59,3.95,4.19,4.19,4.38,4.83,4.73,4.68,4.96,5.32,5.67,5.90,6.50,6.75,7.04,6.80,7.05,7.44,8.10,8.58,8.61,9.00,9.27
Belém,0.50,0.52,0.52,0.56,0.56,0.56,0.57,0.59,0.62,0.62,0.68,0.68,0.72,0.78,0.82,0.83,0.88,1.01,1.02,1.00,1.07,1.16,1.19,1.25,1.28,1.23,1.31,1.22,1.32,1.29,1.41,1.44,1.48,1.57,1.54,1.58,1.73,1.92,2.03,2.15,2.08,2.05,2.09,2.15,2.29,2.22,2.39,2.71
Boa Vista,0.69,0.87,0.95,0.93,0.89,0.96,0.96,0.97,0.90,0.88,0.86,0.89,0.94,0.94,0.98,0.94,1.05,1.12,1.03,1.06,1.10,1.15,1.11,1.19,1.29,1.41,1.53,1.49,1.51,1.34,1.34,1.31,1.20,1.27,1.24,1.24,1.28,1.42,1.54,1.56,1.51,1.63,1.63,1.89,2.01,2.15,2.18,2.10
Brasília,1.48,1.68,1.95,2.05,2.07,2.23,2.31,2.37,2.42,2.47,2.62,2.68,2.78,2.96,3.10,3.07,3.08,3.11,3.29,3.40,3.60,4.02,4.09,4.28,4.42,4.52,5.05,4.92,5.18,5.41,5.52,5.70,5.68,5.95,6.05,6.06,6.28,6.61,6.99,7.24,7.66,7.87,7.99,8.18,8.03,8.24,8.40,8.74
Campo Grande,1.41,1.48,1.62,1.61,1.67,1.92,1.99,2.06,2.18,2.21,2.35,2.52,2.63,2.71,2.71,2.68,2.41,2.46,2.91,2.71,2.85,2.78,2.87,3.05,3.11,3.16,3.20,3.27,3.43,3.51,3.82,3.63,3.61,3.50,3.50,3.49,3.12,3.39,3.35,3.42,3.38,3.47,3.77,3.99,3.85,3.78,4.20,4.31
Cuiabá,0.84,0.91,0.98,1.07,1.26,1.25,1.23,1.37,1.29,1.33,1.45,1.64,1.62,1.71,1.68,1.64,1.62,1.71,1.99,2.01,2.17,2.20,2.29,2.30,2.15,2.06,1.99,1.93,1.98,2.15,2.28,2.34,2.69,2.83,2.97,3.05,3.18,3.44,3.48,3.67,3.58,3.51,3.73,3.96,3.87,3.91,3.87,4.43
Curitiba,2.63,2.90,2.76,2.95,2.95,2.93,2.98,3.03,3.20,3.08,3.40,3.32,3.45,3.55,3.78,3.70,3.50,3.56,3.67,3.51,3.57,3.73,4.05,4.24,4.21,4.64,4.89,5.07,4.91,5.07,5.35,5.32,5.59,5.55,5.99,6.16,6.18,6.56,6.52,6.99,6.53,6.36,6.74,6.63,7.11,7.02,7.37,7.90
Florianópolis,3.92,4.36,5.03,5.27,5.08,5.26,5.33,5.17,5.35,5.60,6.30,6.52,6.16,6.95,7.05,7.37,7.40,7.02,7.56,7.27,7.63,7.92,8.31,8.06,8.25,8.79,9.05,9.06,9.05,9.16,9.75,10.39,10.43,10.46,10.73,12.14,11.96,12.59,13.43,14.02,14.28,12.62,12.51,11.68,10.98,10.64,10.58,12.04
Fortaleza,1.01,1.03,1.09,1.17,1.24,1.35,1.47,1.51,1.53,1.55,1.57,1.53,1.50,1.59,1.81,1.85,1.92,2.05,2.16,2.21,2.29,2.28,2.38,2.43,2.47,2.51,2.68,2.79,2.86,2.93,3.03,3.25,3.06,3.32,3.47,3.48,3.87,3.97,4.48,4.49,4.51,4.47,4.83,4.90,4.84,5.24,5.11,6.01


In [61]:

# 1) garantir tipo igual em todos
IBGE["codigo_ibge_resid"] = IBGE["codigo_ibge_resid"].astype(str)
EmPrEP_Serie_Historica_mun.index = EmPrEP_Serie_Historica_mun.index.astype(str)
Vinculados_mun2.index = Vinculados_mun2.index.astype(str)

# 2) mapa codigo → nome a partir do IBGE
map_mun = (
    IBGE[["codigo_ibge_resid", "nome_mun"]]
    .drop_duplicates("codigo_ibge_resid")
    .set_index("codigo_ibge_resid")["nome_mun"]
)

# 3) lista oficial de codigos do IBGE
codigos_ibge = IBGE["codigo_ibge_resid"].sort_values().unique()

# 4) reindexar serie e vinculados usando os codigos do IBGE
prep = EmPrEP_Serie_Historica_mun.reindex(codigos_ibge)

if "nome_mun" in prep.columns:
    prep = prep.drop(columns="nome_mun")

vinc = Vinculados_mun2.reindex(codigos_ibge)

# 5) calcular indicador
base = prep.div(vinc)
base = base.replace([np.inf, -np.inf], np.nan)
Indicador_mes_mun = base.fillna(0).round(2)

# 6) adicionar nome do municipio
Indicador_mes_mun.insert(
    0,
    "nome_mun",
    map_mun.reindex(Indicador_mes_mun.index)
)

# 7) ordenar colunas de meses
cols_meses = [c for c in Indicador_mes_mun.columns if c != "nome_mun"]

cols_meses_ordenadas = sorted(
    cols_meses,
    key=lambda x: (int(x.split("_")[1]), int(x.split("_")[0]))
)

Indicador_mes_mun = Indicador_mes_mun[["nome_mun"] + cols_meses_ordenadas]
Indicador_mes_mun

,nome_mun,1_2022,2_2022,3_2022,4_2022,5_2022,6_2022,7_2022,8_2022,9_2022,10_2022,11_2022,12_2022,1_2023,2_2023,3_2023,4_2023,5_2023,6_2023,7_2023,8_2023,9_2023,10_2023,11_2023,12_2023,1_2024,2_2024,3_2024,4_2024,5_2024,6_2024,7_2024,8_2024,9_2024,10_2024,11_2024,12_2024,1_2025,2_2025,3_2025,4_2025,5_2025,6_2025,7_2025,8_2025,9_2025,10_2025,11_2025,12_2025
1100015,Alta Floresta D'Oeste,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.50,0.20,0.20,0.20,0.25,0.40,0.25,1.00,1.00,0.50,0.50,0.00,1.00,0.50,0.33,1.00,0.33,0.33
1100023,Ariquemes,0.17,0.18,0.21,0.15,0.15,0.14,0.14,0.33,0.45,0.33,0.33,0.36,0.38,0.30,0.18,0.24,0.28,0.38,0.38,0.37,0.42,0.50,0.60,0.50,0.58,0.50,0.47,0.41,0.21,0.32,0.29,0.37,0.32,0.48,0.39,0.39,0.32,0.60,0.65,0.62,0.80,1.07,1.27,1.29,1.05,1.05,0.85,0.83
1100031,Cabixi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1100049,Cacoal,0.05,0.10,0.11,0.12,0.15,0.20,0.20,0.17,0.22,0.18,0.30,0.42,0.31,0.42,0.38,0.60,0.50,0.82,0.69,0.56,0.40,0.38,0.55,0.62,0.61,0.54,0.64,0.88,1.05,1.00,0.96,1.28,1.44,1.85,1.85,2.50,3.67,1.44,1.73,1.77,1.67,1.81,1.82,2.71,2.71,2.50,2.67,3.09
1100056,Cerejeiras,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.20,0.20,0.25,0.50,0.50,0.33,0.50,0.50,0.33,0.25,0.25,0.00,0.33,0.67,0.00,2.00,3.00,1.50,2.00,1.00,3.00,0.67,1.33,2.00,1.33,1.00,0.75,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,Vianópolis,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.33,0.25,0.25,0.25,0.25,0.33,0.50,0.50,1.00,1.00,1.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5222054,Vicentinópolis,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5222203,Vila Boa,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5222302,Vila Propício,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Salvar no Excel os indicadores atuais

In [66]:
# Cria um aquivo em Excel
book = openpyxl.Workbook()
sheet = book.active
book.save(f"Indicador_PrEP_{mes}_{ano}_V7.xlsx")
writer = pd.ExcelWriter(f"Indicador_PrEP_{mes}_{ano}_V7.xlsx", engine='openpyxl', mode = "a",
                       if_sheet_exists="overlay")
tabela_geral_UF.to_excel(writer, sheet_name="Geral")
tabela_geral_mun.to_excel(writer, sheet_name="Geral", startrow=tabela_geral_UF.shape[0] + 3)
tabela_mun_50k.to_excel(writer, sheet_name="Geral", startrow=tabela_geral_UF.shape[0] + 3 + tabela_geral_mun.shape[0] + 3)
Indicador[['regiao_Res', 'UF_Res', 'codigo_ibge_resid','Município','Populacao_resid','Vinculados', 'Em PrEP','Indicador_Mun','Grupo']].to_excel(writer, sheet_name="Município", index = False)
Indicador[Indicador["Populacao_resid"] >= 50000][['regiao_Res', 'UF_Res', 'codigo_ibge_resid','Município','Populacao_resid','Vinculados', 'Em PrEP','Indicador_Mun','Grupo']].to_excel(writer, sheet_name="Município_50k", index = False)
Indicador_UF.to_excel(writer, sheet_name="UF", index = False)
Indicador_Reg.to_excel(writer, sheet_name="Região", index = False)
Indicador_Nac.to_excel(writer, sheet_name="Nacional", index = False)
Indicador_mes.to_excel(writer, sheet_name="Mensal")
Indicador_mes_mun.to_excel(writer, sheet_name="Mensal_municipio")

writer.close()

In [63]:
csv_file_path1 = f'V:\\2025\\Monitoramento e Avaliação\DOCUMENTOS\\PrEP\\Indicador PrEP HIV\\Indicador_PrEP_mes_mun_{mes}_{ano}_antigo.xlsx'
Indicador_mes_mun.to_csv(csv_file_path1, index=False)

In [64]:
PaRAR

NameError: name 'PaRAR' is not defined

In [ ]:
##Salvando arquivo para power Bi - Painel de Eliminação

# Caminho 2: M: drive
csv_file_path = 'M:\Arquivos Atuais\PowerBI\Eliminacao_HIV\\EmPrEP_PriDisp.csv'
Tabelao_mun.to_csv(csv_file_path, index=False)

csv_file_path2 = 'M:\Arquivos Atuais\PowerBI\Eliminacao_HIV\\PrEP_indicador.csv'
Indicador.to_csv(csv_file_path2, index=False)

### Arquivos excel por UF

In [ ]:
# Dicionário UF código para sigla
codigo_uf_para_sigla = {
    12: 'AC', 27: 'AL', 13: 'AM', 16: 'AP', 29: 'BA', 23: 'CE', 53: 'DF', 32: 'ES',
    52: 'GO', 21: 'MA', 51: 'MT', 50: 'MS', 31: 'MG', 15: 'PA', 25: 'PB', 26: 'PE',
    22: 'PI', 41: 'PR', 33: 'RJ', 24: 'RN', 43: 'RS', 11: 'RO', 14: 'RR', 42: 'SC',
    28: 'SE', 35: 'SP', 17: 'TO'
}

# Caminho para salvar
pasta_saida = r"V:\Monitoramento e Avaliação\DOCUMENTOS\PrEP\Indicador PrEP HIV\Indicador UF"

ufs = Indicador_UF['UF_Res'].unique()

for uf in ufs:
    sigla = codigo_uf_para_sigla.get(uf, str(uf))  # Pega a sigla ou o próprio valor se não achar

    aba1 = Indicador_UF[Indicador_UF['UF_Res'] == uf]
    aba2 = Indicador[
        (Indicador['UF_Res'] == uf) &
        (Indicador["Populacao_resid"] >= 50000)
    ][['UF_Res', 'codigo_ibge_resid','Município','Vinculados', 'Em PrEP','Indicador_Mun','Grupo']]

    nome_arquivo = f"Indicador_PrEP_{sigla}_{mes}_{ano}.xlsx"
    caminho_arquivo = f"{pasta_saida}\\{nome_arquivo}"
    
    with pd.ExcelWriter(caminho_arquivo, engine='xlsxwriter') as writer:
        aba1.to_excel(writer, sheet_name="Indicador_UF", index=False)
        aba2.to_excel(writer, sheet_name="Municípios_50000+", index=False)


In [ ]:
cap_escolha = [
    'Campo Grande',
    'Curitiba',
    'Florianópolis',
    'Fortaleza',
    'Porto Alegre',
    'Brasil'
]

# Filtrar pelo índice (index) do DataFrame
Indicador_filtrado = Indicador_mes.loc[cap_escolha]

# Salvar como Excel
Indicador_filtrado.to_excel(f"Indicador_PrEP_AHA_{mes}_{ano}.xlsx")


In [ ]:
Parar

### Gráficos

In [ ]:
# Seleciona só a linha do Brasil
indicador_brasil = Indicador_mes.loc["Brasil"]

# Prepara os labels e valores
datas = [f"{col.split('_')[1]}-{col.split('_')[0].zfill(2)}" for col in indicador_brasil.index]
datas_ordenadas, valores_ordenados = zip(*sorted(zip(datas, indicador_brasil.values)))

# Função para converter para "jan/2022", "jul/2022", etc.
def formatar_data(data_str):
    ano, mes = data_str.split('-')
    if mes == '01':
        return f"jan/{ano}"
    elif mes == '07':
        return f"jul/{ano}"
    else:
        return f"{mes}/{ano}"

labels_formatados = [formatar_data(d) for d in datas_ordenadas]

# Índices de janeiro e julho (caso queira só esses labels)
indices_marcas = [i for i, d in enumerate(datas_ordenadas) if d.split('-')[1] in ['01', '07']]

plt.figure(figsize=(7,4))
plt.plot(datas_ordenadas, valores_ordenados)
plt.scatter([datas_ordenadas[i] for i in indices_marcas],
            [valores_ordenados[i] for i in indices_marcas],
            color='darkblue', zorder=5, label='Jan/Jul')

# Adiciona os labels nos pontos (formato com vírgula)
for i in indices_marcas:
    plt.text(datas_ordenadas[i], valores_ordenados[i] + 0.05,
             f"{valores_ordenados[i]:.2f}".replace('.', ','), 
             ha='center', va='bottom', fontsize=9)

plt.ylabel('Indicador PrEP:HIV')
plt.title('Indicador PrEP:HIV mensal - Brasil')

# Eixo x com apenas jan/jul e label formatado
plt.xticks([datas_ordenadas[i] for i in indices_marcas], 
           [labels_formatados[i] for i in indices_marcas], rotation=0)

# Eixo y automático
plt.ylim(0, max(valores_ordenados) * 1.15)
plt.tight_layout()
plt.savefig("grafico_BR.png")
plt.show()


In [ ]:
# Gráfico com a série histórica das Regiões
graf = Indicador_mes.tail(6)

plt.figure(figsize=(12, 6))
i = 26
for grupo in graf.index:
    dados = graf.loc[grupo]
    plt.plot(dados.index, dados.values, label=grupo, color = cores[i])
    i = i - 1


# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis = "y", linewidth = 0.1, color = "black")

# Gire os rótulos do eixo x para facilitar a leitura 
x_ticks = dados.index[::6]
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.legend(bbox_to_anchor=(1, 0.8), fancybox=True, reverse = True)
    
plt.show()

# Aproximação dos últimos n meses para avaliação de tendência

n = 18

plt.figure(figsize=(12, 6))

# Adicionando linhas de tendência aos últimos n meses
i = 26
for grupo_tend in graf.index:
    dados_recentes = graf.loc[grupo_tend].tail(n)
    x = np.arange(len(dados_recentes)).reshape(-1, 1)
    y = dados_recentes.values.reshape(-1, 1)

    # Ajuste do modelo de regressão linear
    modelo = LinearRegression().fit(x, y)

    # Predição para os mesmos pontos x
    y_pred = modelo.predict(x)

    # Adicionando a linha de tendência aos últimos 18 meses
    plt.plot(dados_recentes.index, y_pred.flatten(), linestyle='--', label=f'Tendência ({grupo_tend})',color = cores[i])
    i = i - 1

    # Coeficiente angular (inclinação)
    inclinacao = modelo.coef_[0][0]
    
    # Coeficiente de determinação (R²)
    r2 = r2_score(y, y_pred)
    
    # Adicionando anotações ao gráfico
    plt.annotate(f'{grupo_tend} \u03B1 = {inclinacao:.2f} R²: {r2:.2f} ', xy=(dados_recentes.index[-1], dados_recentes.values[-1]), xytext=(5, -5), textcoords='offset points')
    
    
# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis="y", linewidth=0.1, color="black")

# Gire os rótulos do eixo x para facilitar a leitura
x_ticks = dados_recentes.index[::3]  # Ajuste conforme necessário
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# Adicionando rótulos e título ao gráfico
plt.title(f'Linhas de Tendência - Últimos {n} Meses')

for grupo in graf.index:
    dados = graf.loc[grupo].tail(n)
    plt.plot(dados.index, dados.values, label=grupo)

plt.show()

# Teste de Mann-Kendall para avaliar a tendência dos últimos n meses

print(f"Teste de Mann-Kendall para avaliar a tendência dos últimos {n} meses \n")
for grupo_test in graf.index:
    dados_test = graf.loc[grupo_test].tail(n)
    print(f"{grupo_test}: Tendência {mk.original_test(dados_test.values).trend},",
          f"p-value {mk.original_test(dados_test.values).p:.3f},",
          f"Tau {mk.original_test(dados_test.values).Tau:.3f},",
          f"Inclinação {mk.original_test(dados_test.values).slope:.3f}")
    print()


In [ ]:
# Exemplo de construção do dicionário de tendências
resultados_tendencia = {}

for grupo_test in graf.index:
    resultado_mk = mk.original_test(graf.loc[grupo_test].tail(n).values)
    if resultado_mk.trend == "increasing":
        resultados_tendencia[grupo_test] = "increasing"
    elif resultado_mk.trend == "decreasing":
        resultados_tendencia[grupo_test] = "decreasing"
    else:
        resultados_tendencia[grupo_test] = "no trend"


In [ ]:
def gerar_frase_tendencia(resultados):
    nomes_regioes = [k for k in resultados if k != "Brasil"]
    tendencia_brasil = resultados.get("Brasil")
    tendencias_regioes = [resultados[r] for r in nomes_regioes]
    if all(t == tendencia_brasil for t in tendencias_regioes):
        if tendencia_brasil == "increasing":
            return f"Assim como o Brasil, todas as regiões do país apresentaram tendência de aumento na análise de {hoje2}."
        elif tendencia_brasil == "decreasing":
            return f"Assim como o Brasil, todas as regiões do país apresentaram tendência de queda na análise de {hoje2}."
        else:
            return f"Assim como o Brasil, todas as regiões do país não apresentaram tendência significativa na análise de {hoje2}."
    else:
        frases = []
        if tendencia_brasil == "increasing":
            frases.append(f"O Brasil apresentou tendência de aumento.")
        elif tendencia_brasil == "decreasing":
            frases.append(f"O Brasil apresentou tendência de queda.")
        else:
            frases.append(f"O Brasil não apresentou tendência significativa.")

        regioes_alta = [r for r in nomes_regioes if resultados[r] == "increasing"]
        regioes_baixa = [r for r in nomes_regioes if resultados[r] == "decreasing"]
        regioes_estavel = [r for r in nomes_regioes if resultados[r] == "no trend"]

        if regioes_alta:
            frases.append(f"As regiões {', '.join(regioes_alta)} apresentaram tendência de aumento.")
        if regioes_baixa:
            frases.append(f"As regiões {', '.join(regioes_baixa)} apresentaram tendência de queda.")
        if regioes_estavel:
            frases.append(f"As regiões {', '.join(regioes_estavel)} não apresentaram tendência significativa.")
        return " ".join(frases)


In [ ]:
# Gráfico com a série histórica
graf_UF = Indicador_mes.iloc[27:54,:]

plt.figure(figsize=(12, 6))
i = 26
for grupo in graf_UF.index:
    dados = graf_UF.loc[grupo]
    plt.plot(dados.index, dados.values, label=grupo, color = cores[i])
    i = i - 1


# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis = "y", linewidth = 0.1, color = "black")

# Gire os rótulos do eixo x para facilitar a leitura 
x_ticks = dados.index[::6]
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.legend(bbox_to_anchor=(1, 0.8), fancybox=True)
    
plt.show()

# Aproximação dos últimos n meses para avaliação de tendência

n = 12

plt.figure(figsize=(12, 6))
i = 26
# Adicionando linhas de tendência aos últimos n meses
for grupo_tend in graf_UF.index:
    dados_recentes = graf_UF.loc[grupo_tend].tail(n)
    x = np.arange(len(dados_recentes)).reshape(-1, 1)
    y = dados_recentes.values.reshape(-1, 1)

    # Ajuste do modelo de regressão linear
    modelo = LinearRegression().fit(x, y)

    # Predição para os mesmos pontos x
    y_pred = modelo.predict(x)

    # Adicionando a linha de tendência aos últimos 18 meses
    plt.plot(dados_recentes.index, y_pred.flatten(), linestyle='--', label=f'Tendência ({grupo_tend})', color = cores[i])
    i = i - 1

    # Coeficiente angular (inclinação)
    inclinacao = modelo.coef_[0][0]
    
    # Coeficiente de determinação (R²)
    r2 = r2_score(y, y_pred)
    
    # Adicionando anotações ao gráfico
    print(f'{grupo_tend} \u03B1 = {inclinacao:.3f} R²: {r2:.2f}')
    
    
# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis="y", linewidth=0.1, color="black")

# Gire os rótulos do eixo x para facilitar a leitura
x_ticks = dados_recentes.index[::3]  # Ajuste conforme necessário
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# Adicionando rótulos e título ao gráfico
plt.title(f'Linhas de Tendência - Últimos {n} Meses')

plt.show()


# Teste de Mann-Kendall para avaliar a tendência dos últimos n meses

print(f"Teste de Mann-Kendall para avaliar a tendência dos últimos {n} meses \n")
for grupo_test in graf_UF.index:
    dados_test = graf_UF.loc[grupo_test].tail(n)
    print(f"{grupo_test}: Tendência {mk.original_test(dados_test.values).trend},",
          f"p-value {mk.original_test(dados_test.values).p:.3f},",
          f"Tau {mk.original_test(dados_test.values).Tau:.3f},",
          f"Inclinação {mk.original_test(dados_test.values).slope:.3f}")
    print()

In [ ]:
# Vamos pegar o valor mais recente (última coluna) de cada UF
ultimo_mes = graf_UF.columns[-1]
valores_uf = graf_UF[ultimo_mes].copy()

# Pega as duas UFs com maior valor do indicador
top_ufs = valores_uf.sort_values(ascending=False).head(2)

# Formata os nomes e valores (com vírgula como separador decimal)
destaque_ufs = [f"{uf} (razão = {valor:.1f}".replace('.', ',') + ")" for uf, valor in top_ufs.items()]


In [ ]:
# Gráfico com a série histórica das capitais
graf_cap = Indicador_mes.iloc[:27,:]

plt.figure(figsize=(12, 6))
i = 26
for grupo in graf_cap.index:
    dados = graf_cap.loc[grupo]
    plt.plot(dados.index, dados.values, label=grupo, color = cores[i])
    i = i - 1


# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis = "y", linewidth = 0.1, color = "black")

# Gire os rótulos do eixo x para facilitar a leitura 
x_ticks = dados.index[::6]
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.legend(bbox_to_anchor=(1, 0.8), fancybox=True)
    
plt.show()

# Aproximação dos últimos n meses para avaliação de tendência

n = 12

plt.figure(figsize=(12, 6))

# Adicionando linhas de tendência aos últimos n meses
i = 26
for grupo_tend in graf_cap.index:
    dados_recentes = graf_cap.loc[grupo_tend].tail(n)
    x = np.arange(len(dados_recentes)).reshape(-1, 1)
    y = dados_recentes.values.reshape(-1, 1)

    # Ajuste do modelo de regressão linear
    modelo = LinearRegression().fit(x, y)

    # Predição para os mesmos pontos x
    y_pred = modelo.predict(x)

    # Adicionando a linha de tendência aos últimos 18 meses
    plt.plot(dados_recentes.index, y_pred.flatten(), linestyle='--', label=f'Tendência ({grupo_tend})', color = cores[i])
    i = i - 1

    # Coeficiente angular (inclinação)
    inclinacao = modelo.coef_[0][0]
    
    # Coeficiente de determinação (R²)
    r2 = r2_score(y, y_pred)
    
    # Adicionando anotações ao gráfico
    print(f'{grupo_tend} \u03B1 = {inclinacao:.3f} R²: {r2:.2f}')
    
    
# Adicione um grid de fundo
plt.grid(True, linestyle='--', axis="y", linewidth=0.1, color="black")

# Gire os rótulos do eixo x para facilitar a leitura
x_ticks = dados_recentes.index[::3]  # Ajuste conforme necessário
plt.xticks(x_ticks, rotation=45)

# Remova a moldura do gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# Adicionando rótulos e título ao gráfico
plt.title(f'Linhas de Tendência - Últimos {n} Meses')

plt.show()

# Teste de Mann-Kendall para avaliar a tendência dos últimos n meses

print(f"Teste de Mann-Kendall para avaliar a tendência dos últimos {n} meses \n")
for grupo_test in graf_cap.index:
    dados_test = graf_cap.loc[grupo_test].tail(n)
    print(f"{grupo_test}: Tendência {mk.original_test(dados_test.values).trend},",
          f"p-value {mk.original_test(dados_test.values).p:.3f},",
          f"Tau {mk.original_test(dados_test.values).Tau:.3f},",
          f"Inclinação {mk.original_test(dados_test.values).slope:.3f}")
    print()

In [ ]:
from docx import Document
from docx.shared import Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH


# Pega os valores de Y,YY (formata com 2 casas decimais) e o Grupo
Y_YY = f"{Indicador_Nac['Indicador_Nac'].iloc[0]:.2f}".replace(".", ",")
X = Indicador_Nac['Grupo'].iloc[0]

texto = f"""

Segue abaixo o monitoramento do indicador Razão PrEP:Vinculados atualizado para {hoje2}.

Grupos de acordo com o valor do Razão PrEP:HIV:
    Grupo 0: Razão < 1
    Grupo 1: Razão ≥ 1 e < 2
    Grupo 2: Razão ≥ 2 e < 3
    Grupo 3: Razão ≥ 3 e < 4
    Grupo 4: Razão ≥ 4

Lembrando que nossa análise anterior demonstrou que municípios que apresentam Razão PrEP:HIV superior a 3, também possuem taxa mensal crescimento dos novos casos de HIV negativa, ou seja, nesse grupo há redução significativa dos novos casos de HIV no período.
"""

texto_BR = f"O Brasil apresenta uma razão de {Y_YY} e está, portanto, no grupo {X}."
texto_regiao = gerar_frase_tendencia(resultados_tendencia)
texto_uf = "Dentre as UF, destacam-se " + " e ".join(destaque_ufs) + " com os maiores indicadores."
texto_data = f"Banco de dados de {hoje.strftime('%d/%m/%Y')}."

# Criação do documento
tabela_df = tabela_mun_50k.reset_index()
tabela_df = tabela_mun_50k.reset_index().fillna(0)
tabela_df.columns = [''] + list(tabela_df.columns[1:])
doc = Document()
doc.add_paragraph(texto)

# Adicionando a tabela ao documento
table = doc.add_table(rows=1, cols=len(tabela_df.columns))
table.style = 'Light Shading'

# Adicionando o cabeçalho (inclui o nome do índice)
for i, col_name in enumerate(tabela_df.columns):
    table.cell(0, i).text = str(col_name)

# Adicionando as linhas de dados
for _, row in tabela_df.iterrows():
    row_cells = table.add_row().cells
    for i, item in enumerate(row):
        row_cells[i].text = str(item)

doc.add_paragraph('')
doc.add_paragraph(texto_BR)

picture = doc.add_picture("grafico_BR.png", width=Cm(16))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph('')
doc.add_paragraph(texto_regiao)
doc.add_paragraph(texto_uf)
doc.add_paragraph('')
doc.add_paragraph(texto_data)

doc.save(f"Monitoramento_Indicador_PrEP_{mes}_{ano}.docx")
